In [5]:
import pandas as pd
import geopandas as gp
import os
import pdv_functions as pdv

In [6]:
tx_22_vtd_shp = gp.read_file("./raw-from-source/vtds_22g/VTDs_22G.shp")

In [7]:
path = "./raw-from-source/2022-general-vtds-election-data/"

file_holder = []

for file in os.listdir(path):
    if file[0:4] == "2022" and "_VRTO" not in file:
        temp_holder = pd.read_csv(path + file)
        temp_holder["year"] = file[0:4]
        election_type = file.split("_")[2]
        print(election_type)
        temp_holder["election_type"]= election_type
        if election_type in ["Election"]:
            print(file)
            temp_holder["cand_detailed"] = temp_holder["election_type"]+"-:-"+temp_holder["year"]+"-:-"+temp_holder["Office"]+"-:-"+temp_holder["Party"]+"-:-"+temp_holder["Name"]
            temp_holder["mod_vtd"] = temp_holder["FIPS"].astype(str).str.zfill(3) + "-" + temp_holder["VTD"].astype(str)
            temp_holder_pivoted = pd.pivot_table(temp_holder, values=['Votes'], index=["mod_vtd", "County", "FIPS"],columns=["cand_detailed"],aggfunc=sum)
            temp_holder_pivoted.columns = temp_holder_pivoted.columns.droplevel(0)

            file_holder.append(temp_holder_pivoted)

Primary
District
Runoff
Primary
Election
2022_General_Election_Returns.csv
Dist
Runoff


In [8]:
# Combine the files together
joined = pd.concat(file_holder, axis = 1)
joined.reset_index(inplace = True, drop = False)

# Get the race to filter down to appropriate columns
col_name_list = [i.split("-:-")[2] for i in joined.columns if i not in ["mod_vtd", "County", "FIPS"]]

In [9]:
joined

cand_detailed,mod_vtd,County,FIPS,Election-:-2022-:-113th District Judge-:-D-:-Collier,Election-:-2022-:-113th District Judge-:-R-:-Milliron,Election-:-2022-:-117th District Judge-:-D-:-Lopez Leon,Election-:-2022-:-117th District Judge-:-R-:-Barclay,Election-:-2022-:-122nd District Judge-:-D-:-Walsdorf,Election-:-2022-:-122nd District Judge-:-R-:-Jones,Election-:-2022-:-141st District Judge-:-D-:-Klein,...,Election-:-2022-:-U.S. Rep 9-:-R-:-Leon,Election-:-2022-:-Williamson Co Clerk-:-D-:-Smith,Election-:-2022-:-Williamson Co Clerk-:-R-:-Rister,Election-:-2022-:-Williamson Co Judge-:-D-:-Conklin,Election-:-2022-:-Williamson Co Judge-:-L-:-Benedict,Election-:-2022-:-Williamson Co Judge-:-R-:-Gravell,Election-:-2022-:-Williamson Ct at Law 1-:-D-:-McConnell,Election-:-2022-:-Williamson Ct at Law 1-:-R-:-Hallford,Election-:-2022-:-Williamson Ct at Law 3-:-D-:-Velez,Election-:-2022-:-Williamson Ct at Law 3-:-R-:-Arnold
0,001-0001,Anderson,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001-0002,Anderson,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001-0003,Anderson,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001-0004,Anderson,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,001-0005,Anderson,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9629,507-0002,Zavala,507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9630,507-0003,Zavala,507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9631,507-0004,Zavala,507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9632,507-001a,Zavala,507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Filter down to the relevant state races
keep_races =  [i for i in set(col_name_list) if 'President' in i or "U.S. Sen" in i
 or 'Governor' in i or 'Lt. Governor' in i or 'Attorney Gen' in i or 'Comptroller' in i
  or 'Ag Comm' in i or 'Land Comm' in i  or 'RR Comm' in i or 'Sup Ct' in i or 'CCA' in i
    or 'U.S. Rep' in i or 'State Rep' in i or 'State Sen' in i]

In [11]:
not_including = [i for i in col_name_list if i not in keep_races]
not_including.sort()
print(not_including)

['113th District Judge', '113th District Judge', '117th District Judge', '117th District Judge', '122nd District Judge', '122nd District Judge', '141st District Judge', '141st District Judge', '144th District Judge', '144th District Judge', '148th District Judge', '148th District Judge', '150th District Judge', '150th District Judge', '157th District Judge', '157th District Judge', '180th District Judge', '180th District Judge', '182nd District Judge', '182nd District Judge', '183rd District Judge', '183rd District Judge', '184th District Judge', '184th District Judge', '185th District Judge', '185th District Judge', '186th District Judge', '186th District Judge', '187th District Judge', '187th District Judge', '189th District Judge', '189th District Judge', '190th District Judge', '190th District Judge', '208th District Judge', '208th District Judge', '209th District Judge', '209th District Judge', '224th District Judge', '224th District Judge', '225th District Judge', '225th District

In [12]:
# Filter the df down to the relevant columns
keep_races.sort()
keep_names = ["mod_vtd", "County", "FIPS"]
col_names_keep = [i for i in joined.columns if i not in keep_names and i.split("-:-")[2] in keep_races]
joined = joined[keep_names + col_names_keep]

In [13]:
joined

cand_detailed,mod_vtd,County,FIPS,Election-:-2022-:-Ag Comm-:-D-:-Hays,Election-:-2022-:-Ag Comm-:-R-:-Miller,Election-:-2022-:-Attorney Gen-:-D-:-Garza,Election-:-2022-:-Attorney Gen-:-L-:-Ash,Election-:-2022-:-Attorney Gen-:-R-:-Paxton,Election-:-2022-:-CCA 5-:-D-:-Huffman,Election-:-2022-:-CCA 5-:-R-:-Walker,...,Election-:-2022-:-U.S. Rep 5-:-L-:-Hale,Election-:-2022-:-U.S. Rep 5-:-R-:-Gooden,Election-:-2022-:-U.S. Rep 5-:-W-:-Torres,Election-:-2022-:-U.S. Rep 7-:-D-:-Fletcher,Election-:-2022-:-U.S. Rep 7-:-R-:-Teague,Election-:-2022-:-U.S. Rep 8-:-D-:-Jones,Election-:-2022-:-U.S. Rep 8-:-L-:-Eriksen,Election-:-2022-:-U.S. Rep 8-:-R-:-Luttrell,Election-:-2022-:-U.S. Rep 9-:-D-:-Green,Election-:-2022-:-U.S. Rep 9-:-R-:-Leon
0,001-0001,Anderson,1,198.0,607.0,203.0,17.0,592.0,195.0,604.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001-0002,Anderson,1,136.0,1164.0,133.0,27.0,1149.0,134.0,1167.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001-0003,Anderson,1,77.0,567.0,75.0,25.0,551.0,74.0,562.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001-0004,Anderson,1,77.0,792.0,69.0,11.0,796.0,77.0,793.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,001-0005,Anderson,1,10.0,162.0,9.0,3.0,159.0,10.0,161.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9629,507-0002,Zavala,507,440.0,135.0,431.0,6.0,153.0,411.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9630,507-0003,Zavala,507,386.0,91.0,369.0,6.0,108.0,360.0,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9631,507-0004,Zavala,507,239.0,197.0,230.0,13.0,210.0,232.0,188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9632,507-001a,Zavala,507,184.0,86.0,176.0,5.0,90.0,174.0,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def get_election_type(contest):
    first = contest.split("-:-")[0]
    if first == 'Election':
        e_type = "G"
    elif first == "District" or first == "Dist":
        e_type = "S"
    elif first == 'Runoff':
        e_type = "R"
    elif first == 'Primary':
        e_type = "P"
    else:
        print(contest)
        raise ValueError
    return e_type

def get_race(contest):
    mod_level = ""
    level = contest.split("-:-")[2]
    level_change_dict = {
        'President':'PRE',
        "U.S. Sen":'USS',
        'Governor':'GOV',
        'Lt. Governor':'LTG',
        'Attorney Gen':'ATG',
        'Comptroller':'COM',
        'Ag Comm':'AGR',
        'Land Comm':'LAN',
        'RR Comm':'RRC',
        'Sup Ct':'SSC',
        'CCA':'CCA',
        'U.S. Rep':'CON',
        'State Sen':'SU',
        'State Rep':'SL'}
    for val in level_change_dict.keys():
        if val in level:
            mod_level = level_change_dict[val]
    if mod_level == "":
        print(contest)
    if mod_level in ['RRC', 'CCA', 'SSC', "CON", "SU", "SL"]:
            dist_num = str(level.split(" ")[-1])
            if dist_num == "Chief":
                dist_num = 'C'
            if mod_level == "SL":
                fill = 3
            else:
                fill = 2
            mod_level += str(dist_num).zfill(fill)
    return mod_level

def get_party(contest):
    party = contest.split("-:-")[3]
    if party == "W":
        return "O"
    else:
        return party
    
def get_name(contest):
    candidate = contest.split("-:-")[4]
    candidate = candidate.upper()
    candidate = candidate.replace(" ", "")
    likely_last = candidate.split(" ")[-1]
    return likely_last[0:3]

In [15]:
contest_name_change_dict = {}

duplicate_value_list = []
for contest in joined.columns:
    if contest not in keep_names:
        # Add in a condition about the 20
        if "CCA" in contest or "Sup Ct" in contest or "RR Comm" in contest or 'U.S. Rep' in contest or 'State Sen' in contest or 'State Rep' in contest:
            value = get_election_type(contest) + get_race(contest)+ get_party(contest)+ get_name(contest)
            if value in contest_name_change_dict.values():
                duplicate_value_list.append(value)
            contest_name_change_dict[contest] = value 
        else:
            value = get_election_type(contest) + "22" + get_race(contest)+ get_party(contest)+ get_name(contest)
            if value in contest_name_change_dict.values():
                duplicate_value_list.append(value)
            contest_name_change_dict[contest] = value 

if len(duplicate_value_list) > 0:
    print("DUPLICATE VALUES")
    for val in [(k,v) for k, v in contest_name_change_dict.items() if v in duplicate_value_list]:
        print(val)

In [16]:
contest_name_change_dict["Election-:-2022-:-Governor-:-D-:-O'Rourke"] =  "G22GOVDORO"

In [17]:
df = pd.DataFrame([(v, k.split("-:-")[4]) for k, v in contest_name_change_dict.items()], columns=['Candidate', 'Column'])
# # Store the data into a csv file
df.sort_values('Candidate', inplace = True)
df.to_csv('./field_names.csv', sep=',')

In [18]:
pd.DataFrame(contest_name_change_dict.items()).to_csv("./field_names.csv", index = False)

## Add COUNTYFP

In [19]:
joined["COUNTYFP"] = joined["FIPS"].astype(str).str.zfill(3)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_47226/2304954649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined["COUNTYFP"] = joined["FIPS"].astype(str).str.zfill(3)


In [20]:
county_dict = dict(zip(joined["County"].str.upper(), joined["FIPS"].astype(str).str.zfill(3)))

Join to shapefile

In [21]:
tx_22_vtd_shp['mod_vtd'] = tx_22_vtd_shp['CNTY'].astype(str).str.zfill(3)+"-"+tx_22_vtd_shp['VTD'].astype(str)

In [22]:
joined["mod_vtd"] = joined["mod_vtd"].str.upper()

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_47226/1864166198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined["mod_vtd"] = joined["mod_vtd"].str.upper()


In [23]:
tx_22_vtd_comb = gp.GeoDataFrame(pd.merge(joined, tx_22_vtd_shp, how = "outer", on = "mod_vtd", indicator = True))
tx_22_vtd_comb["_merge"].value_counts()

both          9634
left_only        0
right_only       0
Name: _merge, dtype: int64

## Clean columns

In [24]:
tx_22_vtd_comb["UNIQUE_ID"] = tx_22_vtd_comb["mod_vtd"]

In [25]:
sorted_cands = {k: v for k, v in sorted(contest_name_change_dict.items(), key=lambda item: item[1])}

In [26]:
sorted_cands

{'Election-:-2022-:-Ag Comm-:-D-:-Hays': 'G22AGRDHAY',
 'Election-:-2022-:-Ag Comm-:-R-:-Miller': 'G22AGRRMIL',
 'Election-:-2022-:-Attorney Gen-:-D-:-Garza': 'G22ATGDGAR',
 'Election-:-2022-:-Attorney Gen-:-L-:-Ash': 'G22ATGLASH',
 'Election-:-2022-:-Attorney Gen-:-R-:-Paxton': 'G22ATGRPAX',
 'Election-:-2022-:-Comptroller-:-D-:-Dudding': 'G22COMDDUD',
 'Election-:-2022-:-Comptroller-:-L-:-Echevarria-Garza': 'G22COMLECH',
 'Election-:-2022-:-Comptroller-:-R-:-Hegar': 'G22COMRHEG',
 "Election-:-2022-:-Governor-:-D-:-O'Rourke": 'G22GOVDORO',
 'Election-:-2022-:-Governor-:-G-:-Barrios': 'G22GOVGBAR',
 'Election-:-2022-:-Governor-:-L-:-Tippetts': 'G22GOVLTIP',
 'Election-:-2022-:-Governor-:-W-:-Write-In': 'G22GOVOWRI',
 'Election-:-2022-:-Governor-:-R-:-Abbott': 'G22GOVRABB',
 'Election-:-2022-:-Land Comm-:-D-:-Kleberg': 'G22LANDKLE',
 'Election-:-2022-:-Land Comm-:-G-:-Molison': 'G22LANGMOL',
 'Election-:-2022-:-Land Comm-:-W-:-Menger': 'G22LANOMEN',
 'Election-:-2022-:-Land Comm-:-R-:-B

In [27]:
list(sorted_cands.values())

['G22AGRDHAY',
 'G22AGRRMIL',
 'G22ATGDGAR',
 'G22ATGLASH',
 'G22ATGRPAX',
 'G22COMDDUD',
 'G22COMLECH',
 'G22COMRHEG',
 'G22GOVDORO',
 'G22GOVGBAR',
 'G22GOVLTIP',
 'G22GOVOWRI',
 'G22GOVRABB',
 'G22LANDKLE',
 'G22LANGMOL',
 'G22LANOMEN',
 'G22LANRBUC',
 'G22LTGDCOL',
 'G22LTGLSTE',
 'G22LTGRPAT',
 'GCCA05DHUF',
 'GCCA05RWAL',
 'GCCA06DJOH',
 'GCCA06RMCC',
 'GCON01DJEF',
 'GCON01RMOR',
 'GCON02DFUL',
 'GCON02RCRE',
 'GCON03DSRI',
 'GCON03LCLA',
 'GCON03RSEL',
 'GCON04DOME',
 'GCON04LSIM',
 'GCON04RFAL',
 'GCON05DHIL',
 'GCON05LHAL',
 'GCON05OTOR',
 'GCON05RGOO',
 'GCON07DFLE',
 'GCON07RTEA',
 'GCON08DJON',
 'GCON08LERI',
 'GCON08RLUT',
 'GCON09DGRE',
 'GCON09RLEO',
 'GCON10DNUN',
 'GCON10LKEL',
 'GCON10RMCC',
 'GCON12DHUN',
 'GCON12RGRA',
 'GCON13DBRO',
 'GCON13RJAC',
 'GCON14DWIL',
 'GCON14RWEB',
 'GCON15DVAL',
 'GCON15LLEO',
 'GCON15RDEL',
 'GCON16DESC',
 'GCON16RARM',
 'GCON17DWOO',
 'GCON17RSES',
 'GCON18DJAC',
 'GCON18IDUN',
 'GCON18LKUR',
 'GCON18RMON',
 'GCON19ILEW',
 'GCON19RA

In [28]:
tx_22_vtd_comb.rename(columns = contest_name_change_dict, inplace = True)

In [29]:
tx_22_vtd_comb = tx_22_vtd_comb[["UNIQUE_ID","COUNTYFP","County","VTD"]+list(sorted_cands.values()) + ["geometry"]]

In [30]:
tx_22_vtd_comb = tx_22_vtd_comb.fillna(0)
for val in list(sorted_cands.values()):
    tx_22_vtd_comb[val] = tx_22_vtd_comb[val].astype(int)
    #print(val, sum(tx_22_vtd_comb[val]))

In [31]:
## Read in totals to compare against

totals = pd.read_csv("./raw-from-source/Untitled spreadsheet - Voter Turnout Report.csv")

In [32]:
totals["TOTAL VOTES PER OFFICE PER COUNTY"] = totals["TOTAL VOTES PER OFFICE PER COUNTY"].astype(str).str.replace(",","")
totals["TOTAL VOTES PER OFFICE PER COUNTY"] = totals["TOTAL VOTES PER OFFICE PER COUNTY"].astype(int)


In [33]:
totals

,ELECTION DATE-NAME,OFFICE NAME,CANDIDATE NAME,COUNTY NAME,TOTAL VOTES PER OFFICE PER COUNTY
0,11/08/2022 - 2022 NOVEMBER 8TH GENERAL ELECTION,U. S. REPRESENTATIVE DISTRICT 1,JRMAR (JJ) JEFFERSON,BOWIE,5783
1,11/08/2022 - 2022 NOVEMBER 8TH GENERAL ELECTION,U. S. REPRESENTATIVE DISTRICT 1,NATHANIEL MORAN,BOWIE,17229
2,11/08/2022 - 2022 NOVEMBER 8TH GENERAL ELECTION,U. S. REPRESENTATIVE DISTRICT 1,JRMAR (JJ) JEFFERSON,CAMP,846
3,11/08/2022 - 2022 NOVEMBER 8TH GENERAL ELECTION,U. S. REPRESENTATIVE DISTRICT 1,NATHANIEL MORAN,CAMP,3092
4,11/08/2022 - 2022 NOVEMBER 8TH GENERAL ELECTION,U. S. REPRESENTATIVE DISTRICT 1,JRMAR (JJ) JEFFERSON,CASS,1520
...,...,...,...,...,...
11491,11/08/2022 - 2022 NOVEMBER 8TH GENERAL ELECTION,CRIMINAL DISTRICT ATTORNEY MCLENNAN COUNTY,AUBREY ROBERTSON,MCLENNAN,22195
11492,11/08/2022 - 2022 NOVEMBER 8TH GENERAL ELECTION,CRIMINAL DISTRICT ATTORNEY MCLENNAN COUNTY,JOSH TETENS,MCLENNAN,49267
11493,11/08/2022 - 2022 NOVEMBER 8TH GENERAL ELECTION,CRIMINAL DISTRICT ATTORNEY TARRANT COUNTY,TIFFANY D. BURKS,TARRANT,272066
11494,11/08/2022 - 2022 NOVEMBER 8TH GENERAL ELECTION,CRIMINAL DISTRICT ATTORNEY TARRANT COUNTY,PHIL SORRELLS,TARRANT,309730


In [34]:
totals["pivot_col"] = totals["OFFICE NAME"] + "-:-" + totals["CANDIDATE NAME"]

In [35]:
col_list = ['U. S. REPRESENTATIVE DISTRICT 1',
       'U. S. REPRESENTATIVE DISTRICT 2',
       'U. S. REPRESENTATIVE DISTRICT 3',
       'U. S. REPRESENTATIVE DISTRICT 4',
       'U. S. REPRESENTATIVE DISTRICT 5',
       'U. S. REPRESENTATIVE DISTRICT 6',
       'U. S. REPRESENTATIVE DISTRICT 7',
       'U. S. REPRESENTATIVE DISTRICT 8',
       'U. S. REPRESENTATIVE DISTRICT 9',
       'U. S. REPRESENTATIVE DISTRICT 10',
       'U. S. REPRESENTATIVE DISTRICT 11',
       'U. S. REPRESENTATIVE DISTRICT 12',
       'U. S. REPRESENTATIVE DISTRICT 13',
       'U. S. REPRESENTATIVE DISTRICT 14',
       'U. S. REPRESENTATIVE DISTRICT 15',
       'U. S. REPRESENTATIVE DISTRICT 16',
       'U. S. REPRESENTATIVE DISTRICT 17',
       'U. S. REPRESENTATIVE DISTRICT 18',
       'U. S. REPRESENTATIVE DISTRICT 19',
       'U. S. REPRESENTATIVE DISTRICT 20',
       'U. S. REPRESENTATIVE DISTRICT 21',
       'U. S. REPRESENTATIVE DISTRICT 22',
       'U. S. REPRESENTATIVE DISTRICT 23',
       'U. S. REPRESENTATIVE DISTRICT 24',
       'U. S. REPRESENTATIVE DISTRICT 25',
       'U. S. REPRESENTATIVE DISTRICT 26',
       'U. S. REPRESENTATIVE DISTRICT 27',
       'U. S. REPRESENTATIVE DISTRICT 28',
       'U. S. REPRESENTATIVE DISTRICT 29',
       'U. S. REPRESENTATIVE DISTRICT 30',
       'U. S. REPRESENTATIVE DISTRICT 31',
       'U. S. REPRESENTATIVE DISTRICT 32',
       'U. S. REPRESENTATIVE DISTRICT 33',
       'U. S. REPRESENTATIVE DISTRICT 34',
       'U. S. REPRESENTATIVE DISTRICT 35',
       'U. S. REPRESENTATIVE DISTRICT 36',
       'U. S. REPRESENTATIVE DISTRICT 37',
       'U. S. REPRESENTATIVE DISTRICT 38', 'GOVERNOR',
       'LIEUTENANT GOVERNOR', 'ATTORNEY GENERAL',
       'COMPTROLLER OF PUBLIC ACCOUNTS',
       'COMMISSIONER OF THE GENERAL LAND OFFICE',
       'COMMISSIONER OF AGRICULTURE', 'RAILROAD COMMISSIONER ',
       'JUSTICE, SUPREME COURT, PLACE 3',
       'JUSTICE, SUPREME COURT, PLACE 5',
       'JUSTICE, SUPREME COURT, PLACE 9',
       'JUDGE, COURT OF CRIMINAL APPEALS PLACE 2',
       'JUDGE, COURT OF CRIMINAL APPEALS PLACE 5',
       'JUDGE, COURT OF CRIMINAL APPEALS PLACE 6',
       'STATE SENATOR, DISTRICT 1', 'STATE SENATOR, DISTRICT 2',
       'STATE SENATOR, DISTRICT 3', 'STATE SENATOR, DISTRICT 4',
       'STATE SENATOR, DISTRICT 5', 'STATE SENATOR, DISTRICT 6',
       'STATE SENATOR, DISTRICT 7', 'STATE SENATOR, DISTRICT 8',
       'STATE SENATOR, DISTRICT 9', 'STATE SENATOR, DISTRICT 12',
       'STATE SENATOR, DISTRICT 14', 'STATE SENATOR, DISTRICT 15',
       'STATE SENATOR, DISTRICT 16', 'STATE SENATOR, DISTRICT 17',
       'STATE SENATOR, DISTRICT 18', 'STATE SENATOR, DISTRICT 19',
       'STATE SENATOR, DISTRICT 20', 'STATE SENATOR, DISTRICT 21',
       'STATE SENATOR, DISTRICT 22', 'STATE SENATOR, DISTRICT 24',
       'STATE SENATOR, DISTRICT 25', 'STATE SENATOR, DISTRICT 26',
       'STATE SENATOR, DISTRICT 27', 'STATE SENATOR, DISTRICT 28',
       'STATE SENATOR, DISTRICT 29', 'STATE SENATOR, DISTRICT 31',
       'STATE REPRESENTATIVE DISTRICT 4',
       'STATE REPRESENTATIVE DISTRICT 6',
       'STATE REPRESENTATIVE DISTRICT 8',
       'STATE REPRESENTATIVE DISTRICT 9',
       'STATE REPRESENTATIVE DISTRICT 13',
       'STATE REPRESENTATIVE DISTRICT 14',
       'STATE REPRESENTATIVE DISTRICT 15',
       'STATE REPRESENTATIVE DISTRICT 17',
       'STATE REPRESENTATIVE DISTRICT 18',
       'STATE REPRESENTATIVE DISTRICT 19',
       'STATE REPRESENTATIVE DISTRICT 20',
       'STATE REPRESENTATIVE DISTRICT 22',
       'STATE REPRESENTATIVE DISTRICT 23',
       'STATE REPRESENTATIVE DISTRICT 24',
       'STATE REPRESENTATIVE DISTRICT 26',
       'STATE REPRESENTATIVE DISTRICT 27',
       'STATE REPRESENTATIVE DISTRICT 28',
       'STATE REPRESENTATIVE DISTRICT 31',
       'STATE REPRESENTATIVE DISTRICT 33',
       'STATE REPRESENTATIVE DISTRICT 34',
       'STATE REPRESENTATIVE DISTRICT 35',
       'STATE REPRESENTATIVE DISTRICT 37',
       'STATE REPRESENTATIVE DISTRICT 39',
       'STATE REPRESENTATIVE DISTRICT 41',
       'STATE REPRESENTATIVE DISTRICT 42',
       'STATE REPRESENTATIVE DISTRICT 44',
       'STATE REPRESENTATIVE DISTRICT 45',
       'STATE REPRESENTATIVE DISTRICT 46',
       'STATE REPRESENTATIVE DISTRICT 47',
       'STATE REPRESENTATIVE DISTRICT 48',
       'STATE REPRESENTATIVE DISTRICT 49',
       'STATE REPRESENTATIVE DISTRICT 50',
       'STATE REPRESENTATIVE DISTRICT 51',
       'STATE REPRESENTATIVE DISTRICT 52',
       'STATE REPRESENTATIVE DISTRICT 53',
       'STATE REPRESENTATIVE DISTRICT 54',
       'STATE REPRESENTATIVE DISTRICT 55',
       'STATE REPRESENTATIVE DISTRICT 56',
       'STATE REPRESENTATIVE DISTRICT 57',
       'STATE REPRESENTATIVE DISTRICT 61',
       'STATE REPRESENTATIVE DISTRICT 63',
       'STATE REPRESENTATIVE DISTRICT 65',
       'STATE REPRESENTATIVE DISTRICT 66',
       'STATE REPRESENTATIVE DISTRICT 67',
       'STATE REPRESENTATIVE DISTRICT 68',
       'STATE REPRESENTATIVE DISTRICT 69',
       'STATE REPRESENTATIVE DISTRICT 70',
       'STATE REPRESENTATIVE DISTRICT 71',
       'STATE REPRESENTATIVE DISTRICT 73',
       'STATE REPRESENTATIVE DISTRICT 74',
       'STATE REPRESENTATIVE DISTRICT 75',
       'STATE REPRESENTATIVE DISTRICT 76',
       'STATE REPRESENTATIVE DISTRICT 81',
       'STATE REPRESENTATIVE DISTRICT 85',
       'STATE REPRESENTATIVE DISTRICT 86',
       'STATE REPRESENTATIVE DISTRICT 87',
       'STATE REPRESENTATIVE DISTRICT 92',
       'STATE REPRESENTATIVE DISTRICT 93',
       'STATE REPRESENTATIVE DISTRICT 94',
       'STATE REPRESENTATIVE DISTRICT 95',
       'STATE REPRESENTATIVE DISTRICT 97',
       'STATE REPRESENTATIVE DISTRICT 98',
       'STATE REPRESENTATIVE DISTRICT 99',
       'STATE REPRESENTATIVE DISTRICT 100',
       'STATE REPRESENTATIVE DISTRICT 102',
       'STATE REPRESENTATIVE DISTRICT 103',
       'STATE REPRESENTATIVE DISTRICT 105',
       'STATE REPRESENTATIVE DISTRICT 107',
       'STATE REPRESENTATIVE DISTRICT 108',
       'STATE REPRESENTATIVE DISTRICT 111',
       'STATE REPRESENTATIVE DISTRICT 112',
       'STATE REPRESENTATIVE DISTRICT 114',
       'STATE REPRESENTATIVE DISTRICT 115',
       'STATE REPRESENTATIVE DISTRICT 117',
       'STATE REPRESENTATIVE DISTRICT 118',
       'STATE REPRESENTATIVE DISTRICT 119',
       'STATE REPRESENTATIVE DISTRICT 120',
       'STATE REPRESENTATIVE DISTRICT 121',
       'STATE REPRESENTATIVE DISTRICT 122',
       'STATE REPRESENTATIVE DISTRICT 123',
       'STATE REPRESENTATIVE DISTRICT 124',
       'STATE REPRESENTATIVE DISTRICT 125',
       'STATE REPRESENTATIVE DISTRICT 128',
       'STATE REPRESENTATIVE DISTRICT 129',
       'STATE REPRESENTATIVE DISTRICT 131',
       'STATE REPRESENTATIVE DISTRICT 132',
       'STATE REPRESENTATIVE DISTRICT 133',
       'STATE REPRESENTATIVE DISTRICT 134',
       'STATE REPRESENTATIVE DISTRICT 135',
       'STATE REPRESENTATIVE DISTRICT 136',
       'STATE REPRESENTATIVE DISTRICT 137',
       'STATE REPRESENTATIVE DISTRICT 138',
       'STATE REPRESENTATIVE DISTRICT 145',
       'STATE REPRESENTATIVE DISTRICT 148',
       'STATE REPRESENTATIVE DISTRICT 149',
       'STATE REPRESENTATIVE DISTRICT 150']

In [36]:
totals = totals[totals["OFFICE NAME"].isin(col_list)]

In [37]:
totals_pivoted = pd.pivot_table(totals, index = "COUNTY NAME", columns = "pivot_col", values = "TOTAL VOTES PER OFFICE PER COUNTY", aggfunc = sum)

In [38]:
totals_pivoted.reset_index(inplace = True, drop = False)

In [39]:
totals_pivoted = totals_pivoted.fillna(0)

In [40]:
county_dict['LASALLE'] =  '283'

In [41]:
totals_pivoted["COUNTYFP"] = totals_pivoted["COUNTY NAME"].map(county_dict).fillna(totals_pivoted["COUNTY NAME"])

In [42]:
race_cols = [i for i in list(totals_pivoted.columns) if i not in ['COUNTY NAME', 'COUNTYFP']]

In [43]:
# starting_dict = {}
# for race in race_cols:
#     last_name = race.split("-:-")[1].split(" ")[-1].lower()
#     race_count = 0
#     holder = ""
#     for key, val in sorted_cands.items():
#         if last_name in key.lower():
#             race_count  += 1
#             holder = val
#     if race_count  != 1:
#         if race_count == 0:
#             print("NO MATCH")
#             starting_dict[race] = "NO MATCH"
#         else:
#             print("MULTIPLE MATCHES")
#             starting_dict[race] = "MULTIPLES MATCH"
#     else:
#         print("ONE MATCH")
#         starting_dict[race] = holder
#first_run_clean = {k:v for k, v in starting_dict.items() if v not in ["NO MATCH", "MULTIPLES MATCH"]}

In [44]:
first_pass_accurate = {'COMMISSIONER OF AGRICULTURE-:-SUSAN HAYS': 'G22AGRDHAY',
 'COMMISSIONER OF THE GENERAL LAND OFFICE-:-CARRIE EVELYN MENGER': 'G22LANOMEN',
 'COMMISSIONER OF THE GENERAL LAND OFFICE-:-DAWN BUCKINGHAM': 'G22LANRBUC',
 'COMMISSIONER OF THE GENERAL LAND OFFICE-:-JAY KLEBERG': 'G22LANDKLE',
 'COMPTROLLER OF PUBLIC ACCOUNTS-:-JANET T. DUDDING': 'G22COMDDUD',
 'COMPTROLLER OF PUBLIC ACCOUNTS-:-V. ALONZO ECHEVARRIA-GARZA': 'G22COMLECH',
 "GOVERNOR-:-BETO O'ROURKE": 'G22GOVDORO',
 'GOVERNOR-:-DELILAH BARRIOS': 'G22GOVGBAR',
 'GOVERNOR-:-GREG ABBOTT': 'G22GOVRABB',
 'GOVERNOR-:-MARK TIPPETTS': 'G22GOVLTIP',
 'JUSTICE, SUPREME COURT, PLACE 3-:-DEBRA LEHRMANN': 'GSSC03RLEH',
 'JUSTICE, SUPREME COURT, PLACE 3-:-ERIN A. NOWELL': 'GSSC03DNOW',
 'JUSTICE, SUPREME COURT, PLACE 3-:-THOMAS EDWARD OXFORD': 'GSSC03LOXF',
 'JUSTICE, SUPREME COURT, PLACE 5-:-AMANDA REICHEK': 'GSSC05DREI',
 'JUSTICE, SUPREME COURT, PLACE 5-:-REBECA HUDDLE': 'GSSC05RHUD',
 'JUSTICE, SUPREME COURT, PLACE 9-:-EVAN YOUNG': 'GSSC09RYOU',
 'JUSTICE, SUPREME COURT, PLACE 9-:-JULIA MALDONADO': 'GSSC09DMAL',
 'LIEUTENANT GOVERNOR-:-DAN PATRICK': 'G22LTGRPAT',
 'LIEUTENANT GOVERNOR-:-SHANNA STEELE': 'G22LTGLSTE',
 'RAILROAD COMMISSIONER -:-HUNTER WAYNE CROW': 'GRRC01GCRO',
 'RAILROAD COMMISSIONER -:-JAIME ANDRES DIEZ': 'GRRC01LDIE',
 'RAILROAD COMMISSIONER -:-LUKE WARFORD': 'GRRC01DWAR',
 'RAILROAD COMMISSIONER -:-WAYNE CHRISTIAN': 'GRRC01RCHR',
 'STATE REPRESENTATIVE DISTRICT 100-:-JOE ROBERTS': 'GSL100LROB',
 'STATE REPRESENTATIVE DISTRICT 102-:-ANA-MARIA RAMOS': 'GSL102DRAM',
 'STATE REPRESENTATIVE DISTRICT 102-:-SUSAN FISCHER': 'GSL102RFIS',
 'STATE REPRESENTATIVE DISTRICT 103-:-ALEJANDRO ARRIETA': 'GSL103IARR',
 'STATE REPRESENTATIVE DISTRICT 105-:-ALLAN E. MEAGHER': 'GSL105RMEA',
 'STATE REPRESENTATIVE DISTRICT 105-:-TERRY MEZA': 'GSL105DMEZ',
 'STATE REPRESENTATIVE DISTRICT 107-:-SHANE D. NEWSOM': 'GSL107LNEW',
 'STATE REPRESENTATIVE DISTRICT 108-:-ELIZABETH GINSBERG': 'GSL108DGIN',
 'STATE REPRESENTATIVE DISTRICT 108-:-MORGAN MEYER': 'GSL108RMEY',
 'STATE REPRESENTATIVE DISTRICT 111-:-BENNY FLORES YRIGOLLEN': 'GSL111RFLO',
 'STATE REPRESENTATIVE DISTRICT 112-:-ANGIE CHEN BUTTON': 'GSL112RBUT',
 'STATE REPRESENTATIVE DISTRICT 112-:-ELVA CURL': 'GSL112DCUR',
 'STATE REPRESENTATIVE DISTRICT 114-:-JOHN BRYANT': 'GSL114DBRY',
 'STATE REPRESENTATIVE DISTRICT 115-:-MELISA DENIS': 'GSL115RDEN',
 'STATE REPRESENTATIVE DISTRICT 117-:-AARON SCHWOPE': 'GSL117RSCH',
 'STATE REPRESENTATIVE DISTRICT 117-:-PHILIP CORTEZ': 'GSL117DCOR',
 'STATE REPRESENTATIVE DISTRICT 118-:-FRANK RAMIREZ': 'GSL118DRAM',
 'STATE REPRESENTATIVE DISTRICT 118-:-JOHN LUJAN': 'GSL118RLUJ',
 'STATE REPRESENTATIVE DISTRICT 119-:-ELIZABETH  "LIZ" CAMPOS': 'GSL119DCAM',
 'STATE REPRESENTATIVE DISTRICT 120-:-BARBARA GERVIN-HAWKINS': 'GSL120DGER',
 'STATE REPRESENTATIVE DISTRICT 120-:-RONALD PAYNE': 'GSL120RPAY',
 'STATE REPRESENTATIVE DISTRICT 121-:-BECCA MOYER DEFELICE': 'GSL121DMOY',
 'STATE REPRESENTATIVE DISTRICT 121-:-STEVE ALLISON': 'GSL121RALL',
 'STATE REPRESENTATIVE DISTRICT 122-:-ANGI ARAMBURU': 'GSL122DARA',
 'STATE REPRESENTATIVE DISTRICT 122-:-MARK DORAZIO': 'GSL122RDOR',
 'STATE REPRESENTATIVE DISTRICT 122-:-STEPHANIE BERLIN': 'GSL122LBER',
 'STATE REPRESENTATIVE DISTRICT 123-:-CHARLOTTE VALDEZ': 'GSL123RVAL',
 'STATE REPRESENTATIVE DISTRICT 123-:-DIEGO BERNAL': 'GSL123DBER',
 'STATE REPRESENTATIVE DISTRICT 124-:-JOHNNY ARREDONDO': 'GSL124RARR',
 'STATE REPRESENTATIVE DISTRICT 128-:-BRISCOE CAIN': 'GSL128RCAI',
 'STATE REPRESENTATIVE DISTRICT 128-:-CHARLES "CHUCK" CREWS': 'GSL128DCRE',
 'STATE REPRESENTATIVE DISTRICT 129-:-DENNIS PAUL': 'GSL129RPAU',
 'STATE REPRESENTATIVE DISTRICT 129-:-KAT MARVEL': 'GSL129DMAR',
 'STATE REPRESENTATIVE DISTRICT 131-:-ALMA A. ALLEN': 'GSL131DALL',
 'STATE REPRESENTATIVE DISTRICT 131-:-GERRY MONROE': 'GSL131RMON',
 'STATE REPRESENTATIVE DISTRICT 132-:-MIKE SCHOFIELD': 'GSL132RSCH',
 'STATE REPRESENTATIVE DISTRICT 133-:-JAMES PATRICK HARREN': 'GSL133LHAR',
 'STATE REPRESENTATIVE DISTRICT 133-:-MANO DEAYALA': 'GSL133RDEA',
 'STATE REPRESENTATIVE DISTRICT 133-:-MOHAMAD MAAROUF': 'GSL133DMAA',
 'STATE REPRESENTATIVE DISTRICT 134-:-CAROL UNSICKER': 'GSL134LUNS',
 'STATE REPRESENTATIVE DISTRICT 134-:-RYAN MCCONNICO': 'GSL134RMCC',
 'STATE REPRESENTATIVE DISTRICT 135-:-JON ROSENTHAL': 'GSL135DROS',
 'STATE REPRESENTATIVE DISTRICT 135-:-MIKE MAY': 'GSL135RMAY',
 'STATE REPRESENTATIVE DISTRICT 136-:-BURTON CULLEY': 'GSL136LCUL',
 'STATE REPRESENTATIVE DISTRICT 136-:-MICHELLE EVANS': 'GSL136REVA',
 'STATE REPRESENTATIVE DISTRICT 137-:-GENE WU': 'GSL137DWU',
 'STATE REPRESENTATIVE DISTRICT 137-:-LEE SHARP': 'GSL137LSHA',
 'STATE REPRESENTATIVE DISTRICT 138-:-LACEY HULL': 'GSL138RHUL',
 'STATE REPRESENTATIVE DISTRICT 14-:-JOHN RANEY': 'GSL014RRAN',
 'STATE REPRESENTATIVE DISTRICT 145-:-MICHAEL MABRY': 'GSL145RMAB',
 'STATE REPRESENTATIVE DISTRICT 148-:-GRIZZLE TROJACEK': 'GSL148LTRO',
 'STATE REPRESENTATIVE DISTRICT 148-:-KAY SMITH': 'GSL148RSMI',
 'STATE REPRESENTATIVE DISTRICT 149-:-BRAXTON BOGUE': 'GSL149LBOG',
 'STATE REPRESENTATIVE DISTRICT 149-:-HUBERT VO': 'GSL149DVO',
 'STATE REPRESENTATIVE DISTRICT 149-:-LILY TRUONG': 'GSL149RTRU',
 'STATE REPRESENTATIVE DISTRICT 15-:-STEVE TOTH': 'GSL015RTOT',
 'STATE REPRESENTATIVE DISTRICT 150-:-GINNY BROWN DANIEL': 'GSL150DBRO',
 'STATE REPRESENTATIVE DISTRICT 150-:-VALOREE SWANSON': 'GSL150RSWA',
 'STATE REPRESENTATIVE DISTRICT 17-:-LINDA CURTIS': 'GSL017ICUR',
 'STATE REPRESENTATIVE DISTRICT 17-:-MADELINE EDEN': 'GSL017DEDE',
 'STATE REPRESENTATIVE DISTRICT 17-:-STAN GERDES': 'GSL017RGER',
 'STATE REPRESENTATIVE DISTRICT 19-:-ELLEN TROXCLAIR': 'GSL019RTRO',
 'STATE REPRESENTATIVE DISTRICT 20-:-RAUL CAMACHO': 'GSL020DCAM',
 'STATE REPRESENTATIVE DISTRICT 22-:-JACORION RANDLE': 'GSL022RRAN',
 'STATE REPRESENTATIVE DISTRICT 23-:-KEITH G. HENRY': 'GSL023DHEN',
 'STATE REPRESENTATIVE DISTRICT 23-:-TERRI LEO-WILSON': 'GSL023RLEO',
 'STATE REPRESENTATIVE DISTRICT 24-:-GREG BONNEN': 'GSL024RBON',
 'STATE REPRESENTATIVE DISTRICT 24-:-MICHAEL J CREEDON': 'GSL024DCRE',
 'STATE REPRESENTATIVE DISTRICT 24-:-RYAN L. MCCAMY': 'GSL024LMCC',
 'STATE REPRESENTATIVE DISTRICT 26-:-JACEY JETTON': 'GSL026RJET',
 'STATE REPRESENTATIVE DISTRICT 27-:-SOHRAB GILANI': 'GSL027RGIL',
 'STATE REPRESENTATIVE DISTRICT 28-:-GARY GATES': 'GSL028RGAT',
 'STATE REPRESENTATIVE DISTRICT 28-:-NELVIN J.  ADRIATICO': 'GSL028DADR',
 'STATE REPRESENTATIVE DISTRICT 31-:-RYAN GUILLEN': 'GSL031RGUI',
 'STATE REPRESENTATIVE DISTRICT 33-:-GRAESON LYNSKEY': 'GSL033DLYN',
 'STATE REPRESENTATIVE DISTRICT 33-:-JUSTIN HOLLAND': 'GSL033RHOL',
 'STATE REPRESENTATIVE DISTRICT 34-:-ABEL HERRERO': 'GSL034DHER',
 'STATE REPRESENTATIVE DISTRICT 34-:-CAROLYN VAUGHN': 'GSL034RVAU',
 'STATE REPRESENTATIVE DISTRICT 35-:-OSCAR LONGORIA': 'GSL035DLON',
 'STATE REPRESENTATIVE DISTRICT 35-:-OSCAR ROSA': 'GSL035RROS',
 'STATE REPRESENTATIVE DISTRICT 39-:-ARMANDO "MANDO" MARTINEZ': 'GSL039DMAR',
 'STATE REPRESENTATIVE DISTRICT 4-:-MATT SAVINO': 'GSL004LSAV',
 'STATE REPRESENTATIVE DISTRICT 42-:-JOE BRENNAN': 'GSL042RBRE',
 'STATE REPRESENTATIVE DISTRICT 44-:-JOHN KUEMPEL': 'GSL044RKUE',
 'STATE REPRESENTATIVE DISTRICT 44-:-ROBERT M. BOHMFALK': 'GSL044DBOH',
 'STATE REPRESENTATIVE DISTRICT 45-:-ERIN ZWIENER': 'GSL045DZWI',
 'STATE REPRESENTATIVE DISTRICT 46-:-SAM STRASSER': 'GSL046RSTR',
 'STATE REPRESENTATIVE DISTRICT 46-:-SHERYL COLE': 'GSL046DCOL',
 'STATE REPRESENTATIVE DISTRICT 46-:-THOMAS KOST': 'GSL046LKOS',
 'STATE REPRESENTATIVE DISTRICT 47-:-VIKKI GOODWIN': 'GSL047DGOO',
 'STATE REPRESENTATIVE DISTRICT 48-:-DONNA HOWARD': 'GSL048DHOW',
 'STATE REPRESENTATIVE DISTRICT 49-:-J. DAVID ROBERSON': 'GSL049LROB',
 'STATE REPRESENTATIVE DISTRICT 49-:-KATHERINE GRIFFIN': 'GSL049RGRI',
 'STATE REPRESENTATIVE DISTRICT 50-:-JAMES TALARICO': 'GSL050DTAL',
 'STATE REPRESENTATIVE DISTRICT 52-:-LUIS ECHEGARAY': 'GSL052DECH',
 'STATE REPRESENTATIVE DISTRICT 53-:-JOE P. HERRERA': 'GSL053DHER',
 'STATE REPRESENTATIVE DISTRICT 54-:-BRAD BUCKLEY': 'GSL054RBUC',
 'STATE REPRESENTATIVE DISTRICT 54-:-JONATHAN HILDNER': 'GSL054DHIL',
 'STATE REPRESENTATIVE DISTRICT 55-:-HUGH D. SHINE': 'GSL055RSHI',
 'STATE REPRESENTATIVE DISTRICT 55-:-TRISTIAN T.D. SANDERS': 'GSL055DSAN',
 'STATE REPRESENTATIVE DISTRICT 56-:-CHARLES  DOC  ANDERSON': 'GSL056RAND',
 'STATE REPRESENTATIVE DISTRICT 56-:-ERIN SHANK': 'GSL056DSHA',
 'STATE REPRESENTATIVE DISTRICT 57-:-DARREN HAMILTON': 'GSL057LHAM',
 'STATE REPRESENTATIVE DISTRICT 6-:-CODY J. GRACE': 'GSL006DGRA',
 'STATE REPRESENTATIVE DISTRICT 6-:-MATT SCHAEFER': 'GSL006RSCH',
 'STATE REPRESENTATIVE DISTRICT 61-:-FREDERICK FRAZIER': 'GSL061RFRA',
 'STATE REPRESENTATIVE DISTRICT 63-:-BEN BUMGARNER': 'GSL063RBUM',
 'STATE REPRESENTATIVE DISTRICT 63-:-H. DENISE WOOTEN': 'GSL063DWOO',
 'STATE REPRESENTATIVE DISTRICT 65-:-BRITTNEY VERDELL': 'GSL065DVER',
 'STATE REPRESENTATIVE DISTRICT 65-:-KRONDA THIMESCH': 'GSL065RTHI',
 'STATE REPRESENTATIVE DISTRICT 66-:-JESSE RINGNESS': 'GSL066DRIN',
 'STATE REPRESENTATIVE DISTRICT 66-:-MATT SHAHEEN': 'GSL066RSHA',
 'STATE REPRESENTATIVE DISTRICT 67-:-JEFF LEACH': 'GSL067RLEA',
 'STATE REPRESENTATIVE DISTRICT 67-:-KEVIN MORRIS': 'GSL067DMOR',
 'STATE REPRESENTATIVE DISTRICT 69-:-JAMES B. FRANK': 'GSL069RFRA',
 'STATE REPRESENTATIVE DISTRICT 69-:-MICHAEL NEUMANN': 'GSL069LNEU',
 'STATE REPRESENTATIVE DISTRICT 69-:-WALTER COPPAGE': 'GSL069DCOP',
 'STATE REPRESENTATIVE DISTRICT 70-:-JAMEE JOLLY': 'GSL070RJOL',
 'STATE REPRESENTATIVE DISTRICT 70-:-MIHAELA ELIZABETH PLESA': 'GSL070DPLE',
 'STATE REPRESENTATIVE DISTRICT 71-:-LINDA GOOLSBEE': 'GSL071DGOO',
 'STATE REPRESENTATIVE DISTRICT 71-:-STAN LAMBERT': 'GSL071RLAM',
 'STATE REPRESENTATIVE DISTRICT 73-:-CARRIE ISAAC': 'GSL073RISA',
 'STATE REPRESENTATIVE DISTRICT 73-:-JUSTIN CALHOUN': 'GSL073DCAL',
 'STATE REPRESENTATIVE DISTRICT 75-:-JONATHAN MULLINS': 'GSL075LMUL',
 'STATE REPRESENTATIVE DISTRICT 76-:-DAN MATHEWS': 'GSL076RMAT',
 'STATE REPRESENTATIVE DISTRICT 76-:-SULEMAN LALANI': 'GSL076DLAL',
 'STATE REPRESENTATIVE DISTRICT 8-:-R. EDWIN ADAMS': 'GSL008LADA',
 'STATE REPRESENTATIVE DISTRICT 85-:-STAN KITZMAN': 'GSL085RKIT',
 'STATE REPRESENTATIVE DISTRICT 87-:-FOUR PRICE': 'GSL087RPRI',
 'STATE REPRESENTATIVE DISTRICT 87-:-NICK HEARN': 'GSL087LHEA',
 'STATE REPRESENTATIVE DISTRICT 9-:-JASON ROGERS': 'GSL009DROG',
 'STATE REPRESENTATIVE DISTRICT 92-:-JOE F LIVINGSTON': 'GSL092RLIV',
 'STATE REPRESENTATIVE DISTRICT 92-:-SALMAN BHOJANI': 'GSL092DBHO',
 'STATE REPRESENTATIVE DISTRICT 93-:-KC CHOWDHURY': 'GSL093DCHO',
 'STATE REPRESENTATIVE DISTRICT 93-:-NATE SCHATZLINE': 'GSL093RSCH',
 'STATE REPRESENTATIVE DISTRICT 94-:-DENNIS SHERRARD': 'GSL094DSHE',
 'STATE REPRESENTATIVE DISTRICT 94-:-TONY TINDERHOLT': 'GSL094RTIN',
 'STATE REPRESENTATIVE DISTRICT 95-:-TAYLOR MONDICK': 'GSL095RMON',
 'STATE REPRESENTATIVE DISTRICT 97-:-CRAIG GOLDMAN': 'GSL097RGOL',
 'STATE REPRESENTATIVE DISTRICT 97-:-LAURIN MCLAURIN': 'GSL097DMCL',
 'STATE REPRESENTATIVE DISTRICT 98-:-GIOVANNI CAPRIGLIONE': 'GSL098RCAP',
 'STATE REPRESENTATIVE DISTRICT 98-:-SHANNON ELKINS': 'GSL098DELK',
 'STATE REPRESENTATIVE DISTRICT 99-:-CHARLIE GEREN': 'GSL099RGER',
 'STATE REPRESENTATIVE DISTRICT 99-:-MIMI COFFEY': 'GSL099DCOF',
 'STATE SENATOR, DISTRICT 14-:-SARAH ECKHARDT': 'GSU14DECK',
 'STATE SENATOR, DISTRICT 14-:-STEVEN E. HASKETT': 'GSU14LHAS',
 'STATE SENATOR, DISTRICT 15-:-GEORGE BRIAN VACHRIS': 'GSU15RVAC',
 'STATE SENATOR, DISTRICT 15-:-JOHN WHITMIRE': 'GSU15DWHI',
 'STATE SENATOR, DISTRICT 16-:-BRANDON COPELAND': 'GSU16RCOP',
 'STATE SENATOR, DISTRICT 17-:-TITUS BENTON': 'GSU17DBEN',
 'STATE SENATOR, DISTRICT 18-:-JOSH TUTT': 'GSU18DTUT',
 'STATE SENATOR, DISTRICT 18-:-LOIS W. KOLKHORST': 'GSU18RKOL',
 'STATE SENATOR, DISTRICT 2-:-BOB HALL': 'GSU02RHAL',
 'STATE SENATOR, DISTRICT 2-:-PRINCE S. GIADOLOR': 'GSU02DGIA',
 'STATE SENATOR, DISTRICT 20-:-WESTLEY WRIGHT': 'GSU20RWRI',
 'STATE SENATOR, DISTRICT 21-:-ARTHUR DIBIANCA': 'GSU21LDIB',
 'STATE SENATOR, DISTRICT 21-:-JUDITH ZAFFIRINI': 'GSU21DZAF',
 'STATE SENATOR, DISTRICT 21-:-JULIE DAHLBERG': 'GSU21RDAH',
 'STATE SENATOR, DISTRICT 22-:-BRIAN BIRDWELL': 'GSU22RBIR',
 'STATE SENATOR, DISTRICT 22-:-JEREMY SCHROPPEL': 'GSU22LSCH',
 'STATE SENATOR, DISTRICT 24-:-KATHY JONES-HOSPOD': 'GSU24DJON',
 'STATE SENATOR, DISTRICT 25-:-ROBERT WALSH': 'GSU25DWAL',
 'STATE SENATOR, DISTRICT 26-:-ASHTON MURRAY': 'GSU26RMUR',
 'STATE SENATOR, DISTRICT 27-:-MORGAN LAMANTIA': 'GSU27DLAM',
 'STATE SENATOR, DISTRICT 29-:-CÉSAR J. BLANCO': 'GSU29DBLA',
 'STATE SENATOR, DISTRICT 29-:-DEREK L. ZUBELDIA': 'GSU29RZUB',
 'STATE SENATOR, DISTRICT 3-:-DESARAE LINDSEY': 'GSU03LLIN',
 'STATE SENATOR, DISTRICT 3-:-ROBERT NICHOLS': 'GSU03RNIC',
 'STATE SENATOR, DISTRICT 3-:-STEVE RUSSELL': 'GSU03DRUS',
 'STATE SENATOR, DISTRICT 4-:-BRANDON CREIGHTON': 'GSU04RCRE',
 'STATE SENATOR, DISTRICT 4-:-MISTY BISHOP': 'GSU04DBIS',
 'STATE SENATOR, DISTRICT 5-:-CHARLES SCHWERTNER': 'GSU05RSCH',
 'STATE SENATOR, DISTRICT 5-:-TOMMY ESTES': 'GSU05LEST',
 'STATE SENATOR, DISTRICT 8-:-EDWARD KLESS': 'GSU08LKLE',
 'STATE SENATOR, DISTRICT 8-:-JONATHAN COCKS': 'GSU08DCOC',
 'STATE SENATOR, DISTRICT 9-:-GWENN BURUD': 'GSU09DBUR',
 'STATE SENATOR, DISTRICT 9-:-KELLY HANCOCK': 'GSU09RHAN',
 'U. S. REPRESENTATIVE DISTRICT 1-:-JRMAR (JJ) JEFFERSON': 'GCON01DJEF',
 'U. S. REPRESENTATIVE DISTRICT 1-:-NATHANIEL MORAN': 'GCON01RMOR',
 'U. S. REPRESENTATIVE DISTRICT 10-:-BILL KELSEY': 'GCON10LKEL',
 'U. S. REPRESENTATIVE DISTRICT 10-:-LINDA NUNO': 'GCON10DNUN',
 'U. S. REPRESENTATIVE DISTRICT 10-:-MICHAEL MCCAUL': 'GCON10RMCC',
 'U. S. REPRESENTATIVE DISTRICT 12-:-KAY GRANGER': 'GCON12RGRA',
 'U. S. REPRESENTATIVE DISTRICT 14-:-MIKAL WILLIAMS': 'GCON14DWIL',
 'U. S. REPRESENTATIVE DISTRICT 14-:-RANDY WEBER': 'GCON14RWEB',
 'U. S. REPRESENTATIVE DISTRICT 15-:-MICHELLE VALLEJO': 'GCON15DVAL',
 'U. S. REPRESENTATIVE DISTRICT 15-:-MONICA DE LA CRUZ': 'GCON15RDEL',
 'U. S. REPRESENTATIVE DISTRICT 16-:-VERONICA ESCOBAR': 'GCON16DESC',
 'U. S. REPRESENTATIVE DISTRICT 17-:-MARY JO WOODS': 'GCON17DWOO',
 'U. S. REPRESENTATIVE DISTRICT 17-:-PETE SESSIONS': 'GCON17RSES',
 'U. S. REPRESENTATIVE DISTRICT 18-:-CARMEN MARIA MONTIEL': 'GCON18RMON',
 'U. S. REPRESENTATIVE DISTRICT 18-:-PHIL KURTZ': 'GCON18LKUR',
 'U. S. REPRESENTATIVE DISTRICT 18-:-VINCE DUNCAN': 'GCON18IDUN',
 'U. S. REPRESENTATIVE DISTRICT 19-:-JODEY C. ARRINGTON': 'GCON19RARR',
 'U. S. REPRESENTATIVE DISTRICT 19-:-NATHAN LEWIS': 'GCON19ILEW',
 'U. S. REPRESENTATIVE DISTRICT 2-:-DAN CRENSHAW': 'GCON02RCRE',
 'U. S. REPRESENTATIVE DISTRICT 2-:-ROBIN FULFORD': 'GCON02DFUL',
 'U. S. REPRESENTATIVE DISTRICT 20-:-ADAM JONASZ': 'GCON20OJON',
 'U. S. REPRESENTATIVE DISTRICT 20-:-JOAQUIN CASTRO': 'GCON20DCAS',
 'U. S. REPRESENTATIVE DISTRICT 20-:-KYLE SINCLAIR': 'GCON20RSIN',
 'U. S. REPRESENTATIVE DISTRICT 21-:-CLAUDIA ANDREANA ZAPATA': 'GCON21DZAP',
 'U. S. REPRESENTATIVE DISTRICT 22-:-JAMIE KAYE JORDAN': 'GCON22DJOR',
 'U. S. REPRESENTATIVE DISTRICT 22-:-JIM SQUIRES': 'GCON22OSQU',
 'U. S. REPRESENTATIVE DISTRICT 22-:-JOSEPH LEBLANC': 'GCON22LLEB',
 'U. S. REPRESENTATIVE DISTRICT 22-:-TROY E. NEHLS': 'GCON22RNEH',
 'U. S. REPRESENTATIVE DISTRICT 23-:-JOHN LIRA': 'GCON23DLIR',
 'U. S. REPRESENTATIVE DISTRICT 23-:-TONY GONZALES': 'GCON23RGON',
 'U. S. REPRESENTATIVE DISTRICT 24-:-JAN MCDOWELL': 'GCON24DMCD',
 'U. S. REPRESENTATIVE DISTRICT 26-:-MICHAEL C. BURGESS': 'GCON26RBUR',
 'U. S. REPRESENTATIVE DISTRICT 26-:-MIKE KOLLS': 'GCON26LKOL',
 'U. S. REPRESENTATIVE DISTRICT 27-:-MICHAEL CLOUD': 'GCON27RCLO',
 'U. S. REPRESENTATIVE DISTRICT 28-:-HENRY CUELLAR': 'GCON28DCUE',
 'U. S. REPRESENTATIVE DISTRICT 29-:-ROBERT SCHAFRANEK': 'GCON29RSCH',
 'U. S. REPRESENTATIVE DISTRICT 3-:-CHRISTOPHER CLAYTOR': 'GCON03LCLA',
 'U. S. REPRESENTATIVE DISTRICT 3-:-KEITH SELF': 'GCON03RSEL',
 'U. S. REPRESENTATIVE DISTRICT 3-:-SANDEEP SRIVASTAVA': 'GCON03DSRI',
 'U. S. REPRESENTATIVE DISTRICT 30-:-JAMES RODGERS': 'GCON30RROD',
 'U. S. REPRESENTATIVE DISTRICT 30-:-JASMINE  CROCKETT': 'GCON30DCRO',
 'U. S. REPRESENTATIVE DISTRICT 30-:-PHIL GRAY': 'GCON30LGRA',
 'U. S. REPRESENTATIVE DISTRICT 30-:-ZACHARIAH MANNING': 'GCON30IMAN',
 'U. S. REPRESENTATIVE DISTRICT 32-:-ANTONIO SWAD': 'GCON32RSWA',
 'U. S. REPRESENTATIVE DISTRICT 32-:-COLIN ALLRED': 'GCON32DALL',
 'U. S. REPRESENTATIVE DISTRICT 33-:-MARC VEASEY': 'GCON33DVEA',
 'U. S. REPRESENTATIVE DISTRICT 33-:-PATRICK DAVID GILLESPIE': 'GCON33RGIL',
 'U. S. REPRESENTATIVE DISTRICT 34-:-CHRIS ROYAL': 'GCON34IROY',
 'U. S. REPRESENTATIVE DISTRICT 35-:-DAN MCQUEEN': 'GCON35RMCQ',
 'U. S. REPRESENTATIVE DISTRICT 35-:-GREG CASAR': 'GCON35DCAS',
 'U. S. REPRESENTATIVE DISTRICT 36-:-BRIAN BABIN': 'GCON36RBAB',
 'U. S. REPRESENTATIVE DISTRICT 36-:-JON HAIRE': 'GCON36DHAI',
 'U. S. REPRESENTATIVE DISTRICT 37-:-CLARK PATTERSON': 'GCON37LPAT',
 'U. S. REPRESENTATIVE DISTRICT 37-:-JENNY GARCIA SHARON': 'GCON37RGAR',
 'U. S. REPRESENTATIVE DISTRICT 37-:-LLOYD DOGGETT': 'GCON37DDOG',
 'U. S. REPRESENTATIVE DISTRICT 37-:-SHERRI LYNN TAYLOR': 'GCON37OTAY',
 'U. S. REPRESENTATIVE DISTRICT 38-:-DUNCAN F. KLUSSMANN': 'GCON38DKLU',
 'U. S. REPRESENTATIVE DISTRICT 38-:-JOEL DEJEAN': 'GCON38IDEJ',
 'U. S. REPRESENTATIVE DISTRICT 4-:-IRO OMERE': 'GCON04DOME',
 'U. S. REPRESENTATIVE DISTRICT 4-:-JOHN SIMMONS': 'GCON04LSIM',
 'U. S. REPRESENTATIVE DISTRICT 4-:-PAT FALLON': 'GCON04RFAL',
 'U. S. REPRESENTATIVE DISTRICT 5-:-KEVIN A. HALE': 'GCON05LHAL',
 'U. S. REPRESENTATIVE DISTRICT 5-:-LANCE GOODEN': 'GCON05RGOO',
 'U. S. REPRESENTATIVE DISTRICT 5-:-RUTH TORRES': 'GCON05OTOR',
 'U. S. REPRESENTATIVE DISTRICT 5-:-TARTISHA HILL': 'GCON05DHIL',
 'U. S. REPRESENTATIVE DISTRICT 7-:-JOHNNY TEAGUE': 'GCON07RTEA',
 'U. S. REPRESENTATIVE DISTRICT 7-:-LIZZIE FLETCHER': 'GCON07DFLE',
 'U. S. REPRESENTATIVE DISTRICT 8-:-MORGAN LUTTRELL': 'GCON08RLUT',
 'U. S. REPRESENTATIVE DISTRICT 8-:-ROY ERIKSEN': 'GCON08LERI',
 'U. S. REPRESENTATIVE DISTRICT 9-:-AL GREEN': 'GCON09DGRE',
'ATTORNEY GENERAL-:-KEN PAXTON': 'G22ATGRPAX',
 'ATTORNEY GENERAL-:-MARK ASH': 'G22ATGLASH',
 'ATTORNEY GENERAL-:-ROCHELLE MERCEDES GARZA': 'G22ATGDGAR',
 'COMMISSIONER OF AGRICULTURE-:-SID MILLER': 'G22AGRRMIL',
 'COMMISSIONER OF THE GENERAL LAND OFFICE-:-ALFRED MOLISON, JR': 'G22LANGMOL',
 'COMPTROLLER OF PUBLIC ACCOUNTS-:-GLENN HEGAR': 'G22COMRHEG',
 'JUDGE, COURT OF CRIMINAL APPEALS PLACE 5-:-DANA HUFFMAN': 'GCCA05DHUF',
 'JUDGE, COURT OF CRIMINAL APPEALS PLACE 5-:-SCOTT WALKER': 'GCCA05RWAL',
 'JUDGE, COURT OF CRIMINAL APPEALS PLACE 6-:-JESSE F. MCCLURE, III': 'GCCA06RMCC',
 'JUDGE, COURT OF CRIMINAL APPEALS PLACE 6-:-ROBERT JOHNSON': 'GCCA06DJOH',
 'LIEUTENANT GOVERNOR-:-MIKE COLLIER': 'G22LTGDCOL',
                
 'STATE REPRESENTATIVE DISTRICT 100-:-VENTON C. JONES': 'GSL100DJON',
 'STATE REPRESENTATIVE DISTRICT 103-:-RAFAEL M. ANCHÍA': 'GSL103DANC',
 'STATE REPRESENTATIVE DISTRICT 111-:-YVONNE DAVIS': 'GSL111DDAV',
 'STATE REPRESENTATIVE DISTRICT 114-:-SARAH LAMB': 'GSL114RLAM',
 'STATE REPRESENTATIVE DISTRICT 115-:-JULIE JOHNSON': 'GSL115DJOH',
 'STATE REPRESENTATIVE DISTRICT 124-:-JOSEY GARCIA': 'GSL124DGAR',
 'STATE REPRESENTATIVE DISTRICT 125-:-CARLOS ANTONIO RAYMOND': 'GSL125RRAY',
 'STATE REPRESENTATIVE DISTRICT 125-:-RAY LOPEZ': 'GSL125DLOP',
 'STATE REPRESENTATIVE DISTRICT 13-:-ANGELIA ORR': 'GSL013RORR',
 'STATE REPRESENTATIVE DISTRICT 132-:-CAMERON "CAM" CAMPBELL': 'GSL132DCAM',
 'STATE REPRESENTATIVE DISTRICT 134-:-ANN JOHNSON': 'GSL134DJOH',
 'STATE REPRESENTATIVE DISTRICT 138-:-STEPHANIE MORALES': 'GSL138DMOR',
 'STATE REPRESENTATIVE DISTRICT 14-:-JEFF MILLER': 'GSL014LMIL',
 'STATE REPRESENTATIVE DISTRICT 145-:-CHRISTINA MORALES': 'GSL145DMOR',
 'STATE REPRESENTATIVE DISTRICT 148-:-PENNY MORALES SHAW': 'GSL148DMOR',
 'STATE REPRESENTATIVE DISTRICT 19-:-PAM BAGGETT': 'GSL019DBAG',
 'STATE REPRESENTATIVE DISTRICT 26-:-DANIEL LEE': 'GSL026DLEE',
 'STATE REPRESENTATIVE DISTRICT 27-:-RON REYNOLDS': 'GSL027DREY',
 'STATE REPRESENTATIVE DISTRICT 31-:-MARTHA M. GUTIERREZ': 'GSL031DGUT',
 'STATE REPRESENTATIVE DISTRICT 37-:-JANIE LOPEZ': 'GSL037RLOP',
 'STATE REPRESENTATIVE DISTRICT 39-:-JIMMIE GARCIA': 'GSL039RGAR',
 'STATE REPRESENTATIVE DISTRICT 4-:-KEITH BELL': 'GSL004RBEL',
 'STATE REPRESENTATIVE DISTRICT 42-:-RICHARD PEÑA RAYMOND': 'GSL042DRAY',
 'STATE REPRESENTATIVE DISTRICT 45-:-MICHELLE M. LOPEZ': 'GSL045RLOP',
 'STATE REPRESENTATIVE DISTRICT 47-:-ROB MCCARTHY': 'GSL047RMCC',
 'STATE REPRESENTATIVE DISTRICT 48-:-DANIEL JEROME MCCARTHY': 'GSL048LMCC',
 'STATE REPRESENTATIVE DISTRICT 49-:-GINA HINOJOSA': 'GSL049DHIN',
 'STATE REPRESENTATIVE DISTRICT 50-:-VICTOR JOHNSON': 'GSL050RJOH',
 'STATE REPRESENTATIVE DISTRICT 51-:-MARIA LUISA "LULU" FLORES': 'GSL051DFLO',
 'STATE REPRESENTATIVE DISTRICT 51-:-ROBERT REYNOLDS': 'GSL051RREY',
 'STATE REPRESENTATIVE DISTRICT 52-:-CAROLINE HARRIS': 'GSL052RHAR',
 'STATE REPRESENTATIVE DISTRICT 53-:-ANDREW S. MURR': 'GSL053RMUR',
 'STATE REPRESENTATIVE DISTRICT 57-:-RICHARD HAYES': 'GSL057RHAY',
 'STATE REPRESENTATIVE DISTRICT 61-:-SHEENA KING': 'GSL061DKIN',
 'STATE REPRESENTATIVE DISTRICT 74-:-EDDIE MORALES': 'GSL074DMOR',
 'STATE REPRESENTATIVE DISTRICT 74-:-KATHERINE PARKER': 'GSL074RPAR',
 'STATE REPRESENTATIVE DISTRICT 8-:-CODY HARRIS': 'GSL008RHAR',
 'STATE REPRESENTATIVE DISTRICT 85-:-LARRY E. BAGGETT': 'GSL085DBAG',
 'STATE REPRESENTATIVE DISTRICT 85-:-MICHAEL L. MILLER': 'GSL085LMIL',
 'STATE REPRESENTATIVE DISTRICT 9-:-TRENT ASHBY': 'GSL009RASH',
 'STATE REPRESENTATIVE DISTRICT 95-:-NICOLE COLLIER': 'GSL095DCOL',
 'STATE SENATOR, DISTRICT 12-:-FRANCINE LY': 'GSU12DLY',
 'STATE SENATOR, DISTRICT 12-:-TAN PARKER': 'GSU12RPAR',
 'STATE SENATOR, DISTRICT 16-:-NATHAN JOHNSON': 'GSU16DJOH',
 'STATE SENATOR, DISTRICT 17-:-JOAN HUFFMAN': 'GSU17RHUF',
 'STATE SENATOR, DISTRICT 19-:-ROBERT GARZA': 'GSU19RGAR',
 'STATE SENATOR, DISTRICT 19-:-ROLAND GUTIERREZ': 'GSU19DGUT',
 'STATE SENATOR, DISTRICT 24-:-PETE FLORES': 'GSU24RFLO',
 'STATE SENATOR, DISTRICT 25-:-DONNA CAMPBELL': 'GSU25RCAM',
 'STATE SENATOR, DISTRICT 27-:-ADAM HINOJOSA': 'GSU27RHIN',
 'STATE SENATOR, DISTRICT 8-:-ANGELA PAXTON': 'GSU08RPAX',
 'U. S. REPRESENTATIVE DISTRICT 12-:-TREY J. HUNT': 'GCON12DHUN',
 'U. S. REPRESENTATIVE DISTRICT 13-:-KATHLEEN BROWN': 'GCON13DBRO',
 'U. S. REPRESENTATIVE DISTRICT 13-:-RONNY JACKSON': 'GCON13RJAC',
 'U. S. REPRESENTATIVE DISTRICT 16-:-IRENE ARMENDARIZ JACKSON': 'GCON16RARM',
 'U. S. REPRESENTATIVE DISTRICT 18-:-SHEILA JACKSON LEE': 'GCON18DJAC',
 'U. S. REPRESENTATIVE DISTRICT 21-:-CHIP ROY': 'GCON21RROY',
 'U. S. REPRESENTATIVE DISTRICT 28-:-CASSY GARCIA': 'GCON28RGAR',
 'U. S. REPRESENTATIVE DISTRICT 29-:-SYLVIA GARCIA': 'GCON29DGAR',
 'U. S. REPRESENTATIVE DISTRICT 30-:-DEBBIE WALKER': 'GCON30OWAL',
 'U. S. REPRESENTATIVE DISTRICT 33-:-KEN ASHBY': 'GCON33LASH',
 'U. S. REPRESENTATIVE DISTRICT 34-:-MAYRA FLORES': 'GCON34RFLO',
 'U. S. REPRESENTATIVE DISTRICT 34-:-VICENTE GONZALEZ': 'GCON34DGON',
 'U. S. REPRESENTATIVE DISTRICT 38-:-WESLEY HUNT': 'GCON38RHUN',
 'U. S. REPRESENTATIVE DISTRICT 8-:-LAURA JONES': 'GCON08DJON',
 'U. S. REPRESENTATIVE DISTRICT 9-:-JIMMY I. LEON': 'GCON09RLEO',
'STATE REPRESENTATIVE DISTRICT 119-:-ARTHUR M. THOMAS IV':'GSL119LTHO',
 'STATE REPRESENTATIVE DISTRICT 136-:-JOHN H. BUCY III':'GSL136DBUC',
 'STATE REPRESENTATIVE DISTRICT 15-:-KRISTIN JOHNSON':'GSL015DJOH',
 'STATE REPRESENTATIVE DISTRICT 20-:-TERRY M.  WILSON':'GSL020RWIL',
 'STATE REPRESENTATIVE DISTRICT 22-:-CHRISTIAN  "MANUEL" HAYES':'GSL022DHAY',
 'STATE REPRESENTATIVE DISTRICT 37-:-LUIS VILLARREAL JR.':'GSL037DVIL',
 'STATE REPRESENTATIVE DISTRICT 41-:-BOBBY GUERRA':'GSL041DGUE',
 'STATE REPRESENTATIVE DISTRICT 41-:-JOHN (DOC) ROBERT GUERRA':'GSL041RGUE',
 'STATE REPRESENTATIVE DISTRICT 50-:-TED BROWN':'GSL050LBRO',
 'STATE REPRESENTATIVE DISTRICT 75-:-MARY E. GONZÁLEZ':'GSL075DGON',
 'STATE SENATOR, DISTRICT 20-:-JUAN "CHUY" HINOJOSA':'GSU20DHIN',
 'STATE SENATOR, DISTRICT 26-:-JOSÉ MENÉNDEZ':'GSU26DMEN',
 'U. S. REPRESENTATIVE DISTRICT 15-:-ROSS LYNN LEONE, JR.':'GCON15LLEO',
 'U. S. REPRESENTATIVE DISTRICT 23-:-FRANK LOPEZ JR':'GCON23ILOP',
 'U. S. REPRESENTATIVE DISTRICT 27-:-MACLOVIO PEREZ JR.':'GCON27DPER'}


In [45]:
totals_pivoted.rename(columns = first_pass_accurate, inplace = True)

In [46]:
totals_pivoted.columns.value_counts()

COUNTY NAME    1
GSU19RGAR      1
GSU22RBIR      1
GSU21RDAH      1
GSU21DZAF      1
              ..
GSL022RRAN     1
GSL022DHAY     1
GSL020RWIL     1
GSL020DCAM     1
COUNTYFP       1
Name: pivot_col, Length: 383, dtype: int64

In [47]:
set(totals_pivoted.columns).intersection(set(tx_22_vtd_comb))

{'COUNTYFP',
 'G22AGRDHAY',
 'G22AGRRMIL',
 'G22ATGDGAR',
 'G22ATGLASH',
 'G22ATGRPAX',
 'G22COMDDUD',
 'G22COMLECH',
 'G22COMRHEG',
 'G22GOVDORO',
 'G22GOVGBAR',
 'G22GOVLTIP',
 'G22GOVRABB',
 'G22LANDKLE',
 'G22LANGMOL',
 'G22LANOMEN',
 'G22LANRBUC',
 'G22LTGDCOL',
 'G22LTGLSTE',
 'G22LTGRPAT',
 'GCCA05DHUF',
 'GCCA05RWAL',
 'GCCA06DJOH',
 'GCCA06RMCC',
 'GCON01DJEF',
 'GCON01RMOR',
 'GCON02DFUL',
 'GCON02RCRE',
 'GCON03DSRI',
 'GCON03LCLA',
 'GCON03RSEL',
 'GCON04DOME',
 'GCON04LSIM',
 'GCON04RFAL',
 'GCON05DHIL',
 'GCON05LHAL',
 'GCON05OTOR',
 'GCON05RGOO',
 'GCON07DFLE',
 'GCON07RTEA',
 'GCON08DJON',
 'GCON08LERI',
 'GCON08RLUT',
 'GCON09DGRE',
 'GCON09RLEO',
 'GCON10DNUN',
 'GCON10LKEL',
 'GCON10RMCC',
 'GCON12DHUN',
 'GCON12RGRA',
 'GCON13DBRO',
 'GCON13RJAC',
 'GCON14DWIL',
 'GCON14RWEB',
 'GCON15DVAL',
 'GCON15LLEO',
 'GCON15RDEL',
 'GCON16DESC',
 'GCON16RARM',
 'GCON17DWOO',
 'GCON17RSES',
 'GCON18DJAC',
 'GCON18IDUN',
 'GCON18LKUR',
 'GCON18RMON',
 'GCON19ILEW',
 'GCON19RARR

In [48]:
[ i for i in totals_pivoted.columns if i not in tx_22_vtd_comb.columns]

['COUNTY NAME',
 'GOVERNOR-:-JACQUELINE ABERNATHY',
 'GOVERNOR-:-MARK V GOLOBY',
 'JUDGE, COURT OF CRIMINAL APPEALS PLACE 2-:-MARY LOU KEEL',
 'STATE REPRESENTATIVE DISTRICT 107-:-VICTORIA NEAVE CRIADO',
 'STATE REPRESENTATIVE DISTRICT 13-:-CEDRIC DAVIS SR',
 'STATE REPRESENTATIVE DISTRICT 18-:-ERNEST BAILES',
 'STATE REPRESENTATIVE DISTRICT 68-:-DAVID SPILLER',
 'STATE REPRESENTATIVE DISTRICT 81-:-BROOKS LANDGRAF',
 'STATE REPRESENTATIVE DISTRICT 86-:-JOHN T. SMITHEE',
 'STATE SENATOR, DISTRICT 1-:-BRYAN HUGHES',
 'STATE SENATOR, DISTRICT 28-:-CHARLES PERRY',
 'STATE SENATOR, DISTRICT 31-:-KEVIN SPARKS',
 'STATE SENATOR, DISTRICT 6-:-CAROL ALVARADO',
 'STATE SENATOR, DISTRICT 7-:-PAUL BETTENCOURT',
 'U. S. REPRESENTATIVE DISTRICT 11-:-AUGUST PFLUGER',
 'U. S. REPRESENTATIVE DISTRICT 24-:-BETH VANDUYNE',
 'U. S. REPRESENTATIVE DISTRICT 25-:-ROGER WILLIAMS',
 'U. S. REPRESENTATIVE DISTRICT 31-:-JOHN CARTER',
 'U. S. REPRESENTATIVE DISTRICT 6-:-JAKE ELLZEY']

In [49]:
[ i for i in  tx_22_vtd_comb.columns if i not in totals_pivoted.columns]

['UNIQUE_ID',
 'County',
 'VTD',
 'G22GOVOWRI',
 'GCON24RVAN',
 'GSL013DDAV',
 'GSL107DNEA',
 'geometry']

In [50]:
shared_cols = ['G22AGRDHAY',
 'G22AGRRMIL',
 'G22ATGDGAR',
 'G22ATGLASH',
 'G22ATGRPAX',
 'G22COMDDUD',
 'G22COMLECH',
 'G22COMRHEG',
 'G22GOVDORO',
 'G22GOVGBAR',
 'G22GOVLTIP',
 'G22GOVRABB',
 'G22LANDKLE',
 'G22LANGMOL',
 'G22LANOMEN',
 'G22LANRBUC',
 'G22LTGDCOL',
 'G22LTGLSTE',
 'G22LTGRPAT',
 'GCCA05DHUF',
 'GCCA05RWAL',
 'GCCA06DJOH',
 'GCCA06RMCC',
 'GCON01DJEF',
 'GCON01RMOR',
 'GCON02DFUL',
 'GCON02RCRE',
 'GCON03DSRI',
 'GCON03LCLA',
 'GCON03RSEL',
 'GCON04DOME',
 'GCON04LSIM',
 'GCON04RFAL',
 'GCON05DHIL',
 'GCON05LHAL',
 'GCON05OTOR',
 'GCON05RGOO',
 'GCON07DFLE',
 'GCON07RTEA',
 'GCON08DJON',
 'GCON08LERI',
 'GCON08RLUT',
 'GCON09DGRE',
 'GCON09RLEO',
 'GCON10DNUN',
 'GCON10LKEL',
 'GCON10RMCC',
 'GCON12DHUN',
 'GCON12RGRA',
 'GCON13DBRO',
 'GCON13RJAC',
 'GCON14DWIL',
 'GCON14RWEB',
 'GCON15DVAL',
 'GCON15LLEO',
 'GCON15RDEL',
 'GCON16DESC',
 'GCON16RARM',
 'GCON17DWOO',
 'GCON17RSES',
 'GCON18DJAC',
 'GCON18IDUN',
 'GCON18LKUR',
 'GCON18RMON',
 'GCON19ILEW',
 'GCON19RARR',
 'GCON20DCAS',
 'GCON20OJON',
 'GCON20RSIN',
 'GCON21DZAP',
 'GCON21RROY',
 'GCON22DJOR',
 'GCON22LLEB',
 'GCON22OSQU',
 'GCON22RNEH',
 'GCON23DLIR',
 'GCON23ILOP',
 'GCON23RGON',
 'GCON24DMCD',
 'GCON26LKOL',
 'GCON26RBUR',
 'GCON27DPER',
 'GCON27RCLO',
 'GCON28DCUE',
 'GCON28RGAR',
 'GCON29DGAR',
 'GCON29RSCH',
 'GCON30DCRO',
 'GCON30IMAN',
 'GCON30LGRA',
 'GCON30OWAL',
 'GCON30RROD',
 'GCON32DALL',
 'GCON32RSWA',
 'GCON33DVEA',
 'GCON33LASH',
 'GCON33RGIL',
 'GCON34DGON',
 'GCON34IROY',
 'GCON34RFLO',
 'GCON35DCAS',
 'GCON35RMCQ',
 'GCON36DHAI',
 'GCON36RBAB',
 'GCON37DDOG',
 'GCON37LPAT',
 'GCON37OTAY',
 'GCON37RGAR',
 'GCON38DKLU',
 'GCON38IDEJ',
 'GCON38RHUN',
 'GRRC01DWAR',
 'GRRC01GCRO',
 'GRRC01LDIE',
 'GRRC01RCHR',
 'GSL004LSAV',
 'GSL004RBEL',
 'GSL006DGRA',
 'GSL006RSCH',
 'GSL008LADA',
 'GSL008RHAR',
 'GSL009DROG',
 'GSL009RASH',
 'GSL013RORR',
 'GSL014LMIL',
 'GSL014RRAN',
 'GSL015DJOH',
 'GSL015RTOT',
 'GSL017DEDE',
 'GSL017ICUR',
 'GSL017RGER',
 'GSL019DBAG',
 'GSL019RTRO',
 'GSL020DCAM',
 'GSL020RWIL',
 'GSL022DHAY',
 'GSL022RRAN',
 'GSL023DHEN',
 'GSL023RLEO',
 'GSL024DCRE',
 'GSL024LMCC',
 'GSL024RBON',
 'GSL026DLEE',
 'GSL026RJET',
 'GSL027DREY',
 'GSL027RGIL',
 'GSL028DADR',
 'GSL028RGAT',
 'GSL031DGUT',
 'GSL031RGUI',
 'GSL033DLYN',
 'GSL033RHOL',
 'GSL034DHER',
 'GSL034RVAU',
 'GSL035DLON',
 'GSL035RROS',
 'GSL037DVIL',
 'GSL037RLOP',
 'GSL039DMAR',
 'GSL039RGAR',
 'GSL041DGUE',
 'GSL041RGUE',
 'GSL042DRAY',
 'GSL042RBRE',
 'GSL044DBOH',
 'GSL044RKUE',
 'GSL045DZWI',
 'GSL045RLOP',
 'GSL046DCOL',
 'GSL046LKOS',
 'GSL046RSTR',
 'GSL047DGOO',
 'GSL047RMCC',
 'GSL048DHOW',
 'GSL048LMCC',
 'GSL049DHIN',
 'GSL049LROB',
 'GSL049RGRI',
 'GSL050DTAL',
 'GSL050LBRO',
 'GSL050RJOH',
 'GSL051DFLO',
 'GSL051RREY',
 'GSL052DECH',
 'GSL052RHAR',
 'GSL053DHER',
 'GSL053RMUR',
 'GSL054DHIL',
 'GSL054RBUC',
 'GSL055DSAN',
 'GSL055RSHI',
 'GSL056DSHA',
 'GSL056RAND',
 'GSL057LHAM',
 'GSL057RHAY',
 'GSL061DKIN',
 'GSL061RFRA',
 'GSL063DWOO',
 'GSL063RBUM',
 'GSL065DVER',
 'GSL065RTHI',
 'GSL066DRIN',
 'GSL066RSHA',
 'GSL067DMOR',
 'GSL067RLEA',
 'GSL069DCOP',
 'GSL069LNEU',
 'GSL069RFRA',
 'GSL070DPLE',
 'GSL070RJOL',
 'GSL071DGOO',
 'GSL071RLAM',
 'GSL073DCAL',
 'GSL073RISA',
 'GSL074DMOR',
 'GSL074RPAR',
 'GSL075DGON',
 'GSL075LMUL',
 'GSL076DLAL',
 'GSL076RMAT',
 'GSL085DBAG',
 'GSL085LMIL',
 'GSL085RKIT',
 'GSL087LHEA',
 'GSL087RPRI',
 'GSL092DBHO',
 'GSL092RLIV',
 'GSL093DCHO',
 'GSL093RSCH',
 'GSL094DSHE',
 'GSL094RTIN',
 'GSL095DCOL',
 'GSL095RMON',
 'GSL097DMCL',
 'GSL097RGOL',
 'GSL098DELK',
 'GSL098RCAP',
 'GSL099DCOF',
 'GSL099RGER',
 'GSL100DJON',
 'GSL100LROB',
 'GSL102DRAM',
 'GSL102RFIS',
 'GSL103DANC',
 'GSL103IARR',
 'GSL105DMEZ',
 'GSL105RMEA',
 'GSL107LNEW',
 'GSL108DGIN',
 'GSL108RMEY',
 'GSL111DDAV',
 'GSL111RFLO',
 'GSL112DCUR',
 'GSL112RBUT',
 'GSL114DBRY',
 'GSL114RLAM',
 'GSL115DJOH',
 'GSL115RDEN',
 'GSL117DCOR',
 'GSL117RSCH',
 'GSL118DRAM',
 'GSL118RLUJ',
 'GSL119DCAM',
 'GSL119LTHO',
 'GSL120DGER',
 'GSL120RPAY',
 'GSL121DMOY',
 'GSL121RALL',
 'GSL122DARA',
 'GSL122LBER',
 'GSL122RDOR',
 'GSL123DBER',
 'GSL123RVAL',
 'GSL124DGAR',
 'GSL124RARR',
 'GSL125DLOP',
 'GSL125RRAY',
 'GSL128DCRE',
 'GSL128RCAI',
 'GSL129DMAR',
 'GSL129RPAU',
 'GSL131DALL',
 'GSL131RMON',
 'GSL132DCAM',
 'GSL132RSCH',
 'GSL133DMAA',
 'GSL133LHAR',
 'GSL133RDEA',
 'GSL134DJOH',
 'GSL134LUNS',
 'GSL134RMCC',
 'GSL135DROS',
 'GSL135RMAY',
 'GSL136DBUC',
 'GSL136LCUL',
 'GSL136REVA',
 'GSL137DWU',
 'GSL137LSHA',
 'GSL138DMOR',
 'GSL138RHUL',
 'GSL145DMOR',
 'GSL145RMAB',
 'GSL148DMOR',
 'GSL148LTRO',
 'GSL148RSMI',
 'GSL149DVO',
 'GSL149LBOG',
 'GSL149RTRU',
 'GSL150DBRO',
 'GSL150RSWA',
 'GSSC03DNOW',
 'GSSC03LOXF',
 'GSSC03RLEH',
 'GSSC05DREI',
 'GSSC05RHUD',
 'GSSC09DMAL',
 'GSSC09RYOU',
 'GSU02DGIA',
 'GSU02RHAL',
 'GSU03DRUS',
 'GSU03LLIN',
 'GSU03RNIC',
 'GSU04DBIS',
 'GSU04RCRE',
 'GSU05LEST',
 'GSU05RSCH',
 'GSU08DCOC',
 'GSU08LKLE',
 'GSU08RPAX',
 'GSU09DBUR',
 'GSU09RHAN',
 'GSU12DLY',
 'GSU12RPAR',
 'GSU14DECK',
 'GSU14LHAS',
 'GSU15DWHI',
 'GSU15RVAC',
 'GSU16DJOH',
 'GSU16RCOP',
 'GSU17DBEN',
 'GSU17RHUF',
 'GSU18DTUT',
 'GSU18RKOL',
 'GSU19DGUT',
 'GSU19RGAR',
 'GSU20DHIN',
 'GSU20RWRI',
 'GSU21DZAF',
 'GSU21LDIB',
 'GSU21RDAH',
 'GSU22LSCH',
 'GSU22RBIR',
 'GSU24DJON',
 'GSU24RFLO',
 'GSU25DWAL',
 'GSU25RCAM',
 'GSU26DMEN',
 'GSU26RMUR',
 'GSU27DLAM',
 'GSU27RHIN',
 'GSU29DBLA',
 'GSU29RZUB']

In [51]:
holder = pdv.statewide_totals_check(totals_pivoted, "Official Check", tx_22_vtd_comb, "VTD Level", shared_cols)

***Statewide Totals Check***
G22AGRDHAY has a difference of -1095.0 votes
	Official Check: 3473603.0 votes
	VTD Level: 3474698 votes
G22AGRRMIL has a difference of -100.0 votes
	Official Check: 4480186.0 votes
	VTD Level: 4480286 votes
G22ATGDGAR has a difference of -1009.0 votes
	Official Check: 3497267.0 votes
	VTD Level: 3498276 votes
G22ATGLASH has a difference of -18.0 votes
	Official Check: 233750.0 votes
	VTD Level: 233768 votes
G22ATGRPAX has a difference of 127.0 votes
	Official Check: 4278986.0 votes
	VTD Level: 4278859 votes
G22COMDDUD has a difference of -877.0 votes
	Official Check: 3265069.0 votes
	VTD Level: 3265946 votes
G22COMLECH has a difference of -41.0 votes
	Official Check: 212205.0 votes
	VTD Level: 212246 votes
G22COMRHEG has a difference of -64.0 votes
	Official Check: 4496319.0 votes
	VTD Level: 4496383 votes
G22GOVDORO has a difference of -1116.0 votes
	Official Check: 3553656.0 votes
	VTD Level: 3554772 votes
G22GOVGBAR has a difference of -6.0 votes
	Offici

In [53]:
min(holder["Diff"])

-5000.0

In [54]:
max(holder["Diff"])

9968.0

In [55]:
holder[holder["Diff"]==9968.0]

,Race,Official Check_total,VTD Level_total,Diff
106,GSSC03RLEH,4475136.0,4465168,9968.0


In [299]:
pdv.county_totals_check(totals_pivoted, "Official Check", tx_22_vtd_comb, "VTD Level", shared_cols, "COUNTYFP", full_print=False, method='county')

***Countywide Totals Check***

001 contains differences in these races:
	G22LANOMEN has a difference of -4.0 vote(s)
		Official Check: 3.0 vote(s)
		VTD Level: 7 vote(s)
003 contains differences in these races:
	G22LANOMEN has a difference of 5.0 vote(s)
		Official Check: 5.0 vote(s)
		VTD Level: 0 vote(s)
005 contains differences in these races:
	G22LANOMEN has a difference of -7.0 vote(s)
		Official Check: 2.0 vote(s)
		VTD Level: 9 vote(s)
011 contains differences in these races:
	G22AGRRMIL has a difference of 2.0 vote(s)
		Official Check: 826.0 vote(s)
		VTD Level: 824 vote(s)
	G22ATGRPAX has a difference of 2.0 vote(s)
		Official Check: 819.0 vote(s)
		VTD Level: 817 vote(s)
	G22COMRHEG has a difference of 2.0 vote(s)
		Official Check: 820.0 vote(s)
		VTD Level: 818 vote(s)
	G22GOVRABB has a difference of 2.0 vote(s)
		Official Check: 833.0 vote(s)
		VTD Level: 831 vote(s)
	G22LANRBUC has a difference of 2.0 vote(s)
		Official Check: 816.0 vote(s)
		VTD Level: 814 vote(s)
	G22LTG

/Users/peterhorton/Documents/RDH/pber/texas/2022/pdv_functions.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  holder_1 = partner_df.groupby(county_col).sum()
/Users/peterhorton/Documents/RDH/pber/texas/2022/pdv_functions.py:66: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  holder_2 = source_df.groupby(county_col).sum()


027 contains differences in these races:
	G22LANOMEN has a difference of -78.0 vote(s)
		Official Check: 30.0 vote(s)
		VTD Level: 108 vote(s)
029 contains differences in these races:
	G22LANOMEN has a difference of -285.0 vote(s)
		Official Check: 64.0 vote(s)
		VTD Level: 349 vote(s)
	GCON20OJON has a difference of -122.0 vote(s)
		Official Check: 21.0 vote(s)
		VTD Level: 143 vote(s)
035 contains differences in these races:
	G22AGRDHAY has a difference of 4.0 vote(s)
		Official Check: 1088.0 vote(s)
		VTD Level: 1084 vote(s)
	G22AGRRMIL has a difference of 5.0 vote(s)
		Official Check: 6270.0 vote(s)
		VTD Level: 6265 vote(s)
	G22ATGDGAR has a difference of 5.0 vote(s)
		Official Check: 1077.0 vote(s)
		VTD Level: 1072 vote(s)
	G22ATGRPAX has a difference of 6.0 vote(s)
		Official Check: 6082.0 vote(s)
		VTD Level: 6076 vote(s)
	G22COMDDUD has a difference of 5.0 vote(s)
		Official Check: 1019.0 vote(s)
		VTD Level: 1014 vote(s)
	G22COMRHEG has a difference of 5.0 vote(s)
		Official

083 contains differences in these races:
	G22LANOMEN has a difference of -2.0 vote(s)
		Official Check: 0.0 vote(s)
		VTD Level: 2 vote(s)
085 contains differences in these races:
	G22LANOMEN has a difference of -124.0 vote(s)
		Official Check: 73.0 vote(s)
		VTD Level: 197 vote(s)
	GCON04DOME has a difference of 3.0 vote(s)
		Official Check: 43748.0 vote(s)
		VTD Level: 43745 vote(s)
	GCON04RFAL has a difference of 3.0 vote(s)
		Official Check: 48404.0 vote(s)
		VTD Level: 48401 vote(s)
	GCON32DALL has a difference of 7.0 vote(s)
		Official Check: 11459.0 vote(s)
		VTD Level: 11452 vote(s)
	GCON32RSWA has a difference of 2.0 vote(s)
		Official Check: 8225.0 vote(s)
		VTD Level: 8223 vote(s)
	GSL033DLYN has a difference of 2.0 vote(s)
		Official Check: 12136.0 vote(s)
		VTD Level: 12134 vote(s)
	GSU02DGIA has a difference of 2.0 vote(s)
		Official Check: 14826.0 vote(s)
		VTD Level: 14824 vote(s)
	GSU02RHAL has a difference of 1.0 vote(s)
		Official Check: 13266.0 vote(s)
		VTD Level: 

157 contains differences in these races:
	G22LANOMEN has a difference of -89.0 vote(s)
		Official Check: 27.0 vote(s)
		VTD Level: 116 vote(s)
	GCON22OSQU has a difference of -72.0 vote(s)
		Official Check: 126.0 vote(s)
		VTD Level: 198 vote(s)
167 contains differences in these races:
	GCON14DWIL has a difference of -5000.0 vote(s)
		Official Check: 33998.0 vote(s)
		VTD Level: 38998 vote(s)
181 contains differences in these races:
	G22AGRDHAY has a difference of -20.0 vote(s)
		Official Check: 9645.0 vote(s)
		VTD Level: 9665 vote(s)
	G22AGRRMIL has a difference of -44.0 vote(s)
		Official Check: 34718.0 vote(s)
		VTD Level: 34762 vote(s)
	G22ATGDGAR has a difference of -17.0 vote(s)
		Official Check: 9705.0 vote(s)
		VTD Level: 9722 vote(s)
	G22ATGLASH has a difference of -2.0 vote(s)
		Official Check: 1100.0 vote(s)
		VTD Level: 1102 vote(s)
	G22ATGRPAX has a difference of -45.0 vote(s)
		Official Check: 33796.0 vote(s)
		VTD Level: 33841 vote(s)
	G22COMDDUD has a difference of -20

...

249 contains differences in these races:
	G22LANOMEN has a difference of 1.0 vote(s)
		Official Check: 1.0 vote(s)
		VTD Level: 0 vote(s)
253 contains differences in these races:
	G22LANOMEN has a difference of -3.0 vote(s)
		Official Check: 2.0 vote(s)
		VTD Level: 5 vote(s)
257 contains differences in these races:
	G22AGRDHAY has a difference of 21.0 vote(s)
		Official Check: 13581.0 vote(s)
		VTD Level: 13560 vote(s)
	G22AGRRMIL has a difference of 22.0 vote(s)
		Official Check: 28437.0 vote(s)
		VTD Level: 28415 vote(s)
	G22ATGDGAR has a difference of 23.0 vote(s)
		Official Check: 13672.0 vote(s)
		VTD Level: 13649 vote(s)
	G22ATGLASH has a difference of 1.0 vote(s)
		Official Check: 1349.0 vote(s)
		VTD Level: 1348 vote(s)
	G22ATGRPAX has a difference of 20.0 vote(s)
		Official Check: 27377.0 vote(s)
		VTD Level: 27357 vote(s)
	G22COMDDUD has a difference of 21.0 vote(s)
		Official Check: 12922.0 vote(s)
		VTD Level: 12901 vote(s)
	G22COMLECH has a difference of 2.0 vote(s)
		Off

277 contains differences in these races:
	G22AGRDHAY has a difference of -7.0 vote(s)
		Official Check: 2649.0 vote(s)
		VTD Level: 2656 vote(s)
	G22AGRRMIL has a difference of -25.0 vote(s)
		Official Check: 12440.0 vote(s)
		VTD Level: 12465 vote(s)
	G22ATGDGAR has a difference of -8.0 vote(s)
		Official Check: 2627.0 vote(s)
		VTD Level: 2635 vote(s)
	G22ATGRPAX has a difference of -25.0 vote(s)
		Official Check: 12237.0 vote(s)
		VTD Level: 12262 vote(s)
	G22COMDDUD has a difference of -6.0 vote(s)
		Official Check: 2512.0 vote(s)
		VTD Level: 2518 vote(s)
	G22COMRHEG has a difference of -25.0 vote(s)
		Official Check: 12433.0 vote(s)
		VTD Level: 12458 vote(s)
	G22GOVDORO has a difference of -8.0 vote(s)
		Official Check: 2657.0 vote(s)
		VTD Level: 2665 vote(s)
	G22GOVRABB has a difference of -25.0 vote(s)
		Official Check: 12521.0 vote(s)
		VTD Level: 12546 vote(s)
	G22LANDKLE has a difference of -6.0 vote(s)
		Official Check: 2564.0 vote(s)
		VTD Level: 2570 vote(s)
	G22LANGMOL

337 contains differences in these races:
	G22LANOMEN has a difference of -5.0 vote(s)
		Official Check: 0.0 vote(s)
		VTD Level: 5 vote(s)
341 contains differences in these races:
	G22LANOMEN has a difference of -2.0 vote(s)
		Official Check: 0.0 vote(s)
		VTD Level: 2 vote(s)
343 contains differences in these races:
	G22LANOMEN has a difference of -1.0 vote(s)
		Official Check: 1.0 vote(s)
		VTD Level: 2 vote(s)
349 contains differences in these races:
	G22GOVRABB has a difference of 1.0 vote(s)
		Official Check: 10830.0 vote(s)
		VTD Level: 10829 vote(s)
353 contains differences in these races:
	G22LANOMEN has a difference of -2.0 vote(s)
		Official Check: 2.0 vote(s)
		VTD Level: 4 vote(s)
	G22LTGDCOL has a difference of -50.0 vote(s)
		Official Check: 610.0 vote(s)
		VTD Level: 660 vote(s)
361 contains differences in these races:
	G22LANOMEN has a difference of -7.0 vote(s)
		Official Check: 5.0 vote(s)
		VTD Level: 12 vote(s)
365 contains differences in these races:
	G22LANOMEN ha

	GSSC03DNOW has a difference of 10.0 vote(s)
		Official Check: 8364.0 vote(s)
		VTD Level: 8354 vote(s)
	GSSC03RLEH has a difference of 55.0 vote(s)
		Official Check: 29732.0 vote(s)
		VTD Level: 29677 vote(s)
	GSSC05DREI has a difference of 9.0 vote(s)
		Official Check: 8731.0 vote(s)
		VTD Level: 8722 vote(s)
	GSSC05RHUD has a difference of 56.0 vote(s)
		Official Check: 30156.0 vote(s)
		VTD Level: 30100 vote(s)
	GSSC09DMAL has a difference of 9.0 vote(s)
		Official Check: 8866.0 vote(s)
		VTD Level: 8857 vote(s)
	GSSC09RYOU has a difference of 56.0 vote(s)
		Official Check: 29984.0 vote(s)
		VTD Level: 29928 vote(s)
453 contains differences in these races:
	G22AGRDHAY has a difference of -1084.0 vote(s)
		Official Check: 324725.0 vote(s)
		VTD Level: 325809 vote(s)
	G22AGRRMIL has a difference of -279.0 vote(s)
		Official Check: 124923.0 vote(s)
		VTD Level: 125202 vote(s)
	G22ATGDGAR has a difference of -1112.0 vote(s)
		Official Check: 329782.0 vote(s)
		VTD Level: 330894 vote(s)

	GSSC03RLEH has a difference of 50.0 vote(s)
		Official Check: 17622.0 vote(s)
		VTD Level: 17572 vote(s)
	GSSC09DMAL has a difference of 1.0 vote(s)
		Official Check: 2394.0 vote(s)
		VTD Level: 2393 vote(s)
469 contains differences in these races:
	G22LANOMEN has a difference of -9.0 vote(s)
		Official Check: 3.0 vote(s)
		VTD Level: 12 vote(s)
471 contains differences in these races:
	G22LANOMEN has a difference of 1.0 vote(s)
		Official Check: 1.0 vote(s)
		VTD Level: 0 vote(s)
475 contains differences in these races:
	G22LTGLSTE has a difference of -27.0 vote(s)
		Official Check: 32.0 vote(s)
		VTD Level: 59 vote(s)
477 contains differences in these races:
	G22LANOMEN has a difference of -1.0 vote(s)
		Official Check: 2.0 vote(s)
		VTD Level: 3 vote(s)
479 contains differences in these races:
	G22LANOMEN has a difference of -42.0 vote(s)
		Official Check: 17.0 vote(s)
		VTD Level: 59 vote(s)
	GSL042DRAY has a difference of 11.0 vote(s)
		Official Check: 24075.0 vote(s)
		VTD Level

In [47]:
tx_22_vtd_comb.rename(columns = {"VTD":"TX_VTD"}, inplace = True)

In [49]:
tx_22_vtd_comb.to_file("./tx_2022_gen_tx_vtd/tx_2022_gen_tx_vtd.shp", index = False)

In [ ]:
tx_22_vtd_comb